<a href="https://colab.research.google.com/github/MadmanMarble/Planet_U-NET/blob/main/mosaic_probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 44.1 MB/s eta 0:00:00


In [4]:
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.utils import normalize
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import Callback
import rasterio
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras.models import load_model
import pickle
import numpy as np
from tensorflow import keras
from PIL import Image


In [243]:
# for 1 year images - this with handling of band 5
def read_multi_image_withfive(file_path):
    with rasterio.open(file_path) as src:
      bands = [src.read(i) for i in range(1,5)]

      image = np.stack(bands, axis=-1)

      image = image.astype(float)
      max_value = image.max(axis=(0,1), keepdims=True)
      min_value = image.min(axis=(0,1), keepdims=True)
      image = (image - min_value) / (max_value - min_value)

      band_five = src.read(5)

      band_five = band_five.astype(float) /band_five.max()
      band_five = np.expand_dims(band_five, axis=-1)
      combined_image = np.concatenate([image, band_five], axis=-1)


      return combined_image

In [222]:
directory_path = "/content/drive/MyDrive/data/images/kahuku/kahuku_3year"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

There are 48 files in /content/drive/MyDrive/data/images/kahuku/kahuku_3year.


In [244]:
model_01 = keras.models.load_model('/content/drive/MyDrive/data/model_prediction/oneyear_data/20240418_1448/20240418_1448.h5')
model_02 = keras.models.load_model('/content/drive/MyDrive/data/model_prediction/twoyear_data/20240418_1448/20240418_1448_2.h5')
model_03 = keras.models.load_model("/content/drive/MyDrive/data/model_prediction/threeyear_data/20240418_1448/20240418_1448_3.h5")

In [266]:
kawainui_3year = sorted(glob.glob(f"/content/drive/MyDrive/data/images/kawainui/kawainui_3year/*.tif"))
destination = '/content/kawainui_3year'
os.makedirs(destination, exist_ok=True)
for file in kawainui_3year:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/kawainui_3year"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

Copied /content/drive/MyDrive/data/images/kawainui/kawainui_3year/1465_3year.tif to /content/kawainui_3year/1465_3year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_3year/1466_3year.tif to /content/kawainui_3year/1466_3year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_3year/1472_3year.tif to /content/kawainui_3year/1472_3year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_3year/1481_3year.tif to /content/kawainui_3year/1481_3year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_3year/1482_3year.tif to /content/kawainui_3year/1482_3year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_3year/1483_3year.tif to /content/kawainui_3year/1483_3year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_3year/1488_3year.tif to /content/kawainui_3year/1488_3year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_3year/1489_3year.tif to /content/kawainui_3year/1489_3year.tif
Copied /content/

In [256]:
kawainui_2year = sorted(glob.glob(f"/content/drive/MyDrive/data/images/kawainui/kawainui_2year/*.tif"))
destination = '/content/kawainui_2year'
os.makedirs(destination, exist_ok=True)
for file in kawainui_2year:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/kawainui_2year"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

Copied /content/drive/MyDrive/data/images/kawainui/kawainui_2year/1465_2year.tif to /content/kawainui_2year/1465_2year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_2year/1466_2year.tif to /content/kawainui_2year/1466_2year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_2year/1472_2year.tif to /content/kawainui_2year/1472_2year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_2year/1481_2year.tif to /content/kawainui_2year/1481_2year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_2year/1482_2year.tif to /content/kawainui_2year/1482_2year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_2year/1483_2year.tif to /content/kawainui_2year/1483_2year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_2year/1488_2year.tif to /content/kawainui_2year/1488_2year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_2year/1489_2year.tif to /content/kawainui_2year/1489_2year.tif
Copied /content/

In [250]:
kawainui_1year = sorted(glob.glob(f"/content/drive/MyDrive/data/images/kawainui/kawainui_1year/*.tif"))
destination = '/content/kawainui_1year'
os.makedirs(destination, exist_ok=True)
for file in kawainui_1year:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/kawainui_1year"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

Copied /content/drive/MyDrive/data/images/kawainui/kawainui_1year/1465_1year.tif to /content/kawainui_1year/1465_1year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_1year/1466_1year.tif to /content/kawainui_1year/1466_1year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_1year/1472_1year.tif to /content/kawainui_1year/1472_1year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_1year/1481_1year.tif to /content/kawainui_1year/1481_1year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_1year/1482_1year.tif to /content/kawainui_1year/1482_1year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_1year/1483_1year.tif to /content/kawainui_1year/1483_1year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_1year/1488_1year.tif to /content/kawainui_1year/1488_1year.tif
Copied /content/drive/MyDrive/data/images/kawainui/kawainui_1year/1489_1year.tif to /content/kawainui_1year/1489_1year.tif
Copied /content/

In [242]:
kahuku_1year = sorted(glob.glob(f"/content/drive/MyDrive/data/images/kahuku/kahuku_1year/*.tif"))
destination = '/content/kahuku_1year'
os.makedirs(destination, exist_ok=True)
for file in kahuku_1year:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/kahuku_1year"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

Copied /content/drive/MyDrive/data/images/kahuku/kahuku_1year/742_1year.tif to /content/kahuku_1year/742_1year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_1year/743_1year.tif to /content/kahuku_1year/743_1year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_1year/759_1year.tif to /content/kahuku_1year/759_1year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_1year/760_1year.tif to /content/kahuku_1year/760_1year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_1year/761_1year.tif to /content/kahuku_1year/761_1year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_1year/762_1year.tif to /content/kahuku_1year/762_1year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_1year/776_1year.tif to /content/kahuku_1year/776_1year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_1year/777_1year.tif to /content/kahuku_1year/777_1year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_1year/778_1year.tif to /

In [235]:
kahuku_2year = sorted(glob.glob(f"/content/drive/MyDrive/data/images/kahuku/kahuku_2year/*.tif"))
destination = '/content/kahuku_2year'
os.makedirs(destination, exist_ok=True)
for file in kahuku_2year:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/kahuku_2year"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

Copied /content/drive/MyDrive/data/images/kahuku/kahuku_2year/742_2year.tif to /content/kahuku_2year/742_2year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_2year/743_2year.tif to /content/kahuku_2year/743_2year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_2year/759_2year.tif to /content/kahuku_2year/759_2year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_2year/760_2year.tif to /content/kahuku_2year/760_2year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_2year/761_2year.tif to /content/kahuku_2year/761_2year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_2year/762_2year.tif to /content/kahuku_2year/762_2year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_2year/776_2year.tif to /content/kahuku_2year/776_2year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_2year/777_2year.tif to /content/kahuku_2year/777_2year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_2year/778_2year.tif to /

In [223]:
kahuku_3year = sorted(glob.glob(f"/content/drive/MyDrive/data/images/kahuku/kahuku_3year/*.tif"))
destination = '/content/kahuku_3year'
os.makedirs(destination, exist_ok=True)
for file in kahuku_3year:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/kahuku_3year"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

Copied /content/drive/MyDrive/data/images/kahuku/kahuku_3year/742_3year.tif to /content/kahuku_3year/742_3year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_3year/743_3year.tif to /content/kahuku_3year/743_3year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_3year/759_3year.tif to /content/kahuku_3year/759_3year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_3year/760_3year.tif to /content/kahuku_3year/760_3year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_3year/761_3year.tif to /content/kahuku_3year/761_3year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_3year/762_3year.tif to /content/kahuku_3year/762_3year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_3year/776_3year.tif to /content/kahuku_3year/776_3year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_3year/777_3year.tif to /content/kahuku_3year/777_3year.tif
Copied /content/drive/MyDrive/data/images/kahuku/kahuku_3year/778_3year.tif to /

In [166]:
haleiwa_03 = sorted(glob.glob(f"/content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/*.tif"))
destination = '/content/haleiwa_03'
os.makedirs(destination, exist_ok=True)
for file in haleiwa_03:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/haleiwa_03"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/111_3year.tif to /content/haleiwa_03/111_3year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/112_3year.tif to /content/haleiwa_03/112_3year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/126_3year.tif to /content/haleiwa_03/126_3year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/127_3year.tif to /content/haleiwa_03/127_3year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/140_3year.tif to /content/haleiwa_03/140_3year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/141_3year.tif to /content/haleiwa_03/141_3year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/142_3year.tif to /content/haleiwa_03/142_3year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/143_3year.tif to /content/haleiwa_03/143_3year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_3year/154_3year.tif to

In [108]:
haleiwa_02 = sorted(glob.glob(f"/content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/*.tif"))
destination = '/content/haleiwa_02'
os.makedirs(destination, exist_ok=True)
for file in haleiwa_02:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/haleiwa_02"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")


Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/111_2year.tif to /content/haleiwa_02/111_2year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/112_2year.tif to /content/haleiwa_02/112_2year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/126_2year.tif to /content/haleiwa_02/126_2year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/127_2year.tif to /content/haleiwa_02/127_2year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/140_2year.tif to /content/haleiwa_02/140_2year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/141_2year.tif to /content/haleiwa_02/141_2year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/142_2year.tif to /content/haleiwa_02/142_2year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/143_2year.tif to /content/haleiwa_02/143_2year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_2year/154_2year.tif to

In [7]:
haleiwa_01 = sorted(glob.glob(f"/content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/*.tif"))

destination = '/content/haleiwa_01'
os.makedirs(destination, exist_ok=True)
for file in haleiwa_01:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/haleiwa_01"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/111_1year.tif to /content/haleiwa_01/111_1year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/112_1year.tif to /content/haleiwa_01/112_1year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/126_1year.tif to /content/haleiwa_01/126_1year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/127_1year.tif to /content/haleiwa_01/127_1year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/140_1year.tif to /content/haleiwa_01/140_1year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/141_1year.tif to /content/haleiwa_01/141_1year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/142_1year.tif to /content/haleiwa_01/142_1year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/143_1year.tif to /content/haleiwa_01/143_1year.tif
Copied /content/drive/MyDrive/data/images/haleiwa/haleiwa_1year/154_1year.tif to

In [8]:
hale_01 = [read_multi_image_withfive(file) for file in haleiwa_01]

In [ ]:
hale_01

In [9]:
haleiwa_01 = np.array(hale_01)

In [ ]:
haleiwa_01

In [10]:
haleiwa_01.shape

(42, 384, 384, 5)

In [11]:
haleiwa_01_prediction = model_01.predict(haleiwa_01)

2/2 [==============================] - 145s 31s/step


In [12]:
input_01_path = "/content/haleiwa_01/"
output_01_path = "/content/haleiwa_01_output/"
output_weight_01_path = "/content/haleiwa_01_output_weight/"
os.makedirs(input_01_path, exist_ok=True)
os.makedirs(output_01_path, exist_ok=True)
os.makedirs(output_weight_01_path, exist_ok=True)

In [13]:
def create_weight_matrix(height, width):
    center_x, center_y = width // 2, height // 2
    Y, X = np.ogrid[:height, :width]
    distance_from_center = np.sqrt((X - center_x)**2 + (Y - center_y)**2)

    # Normalize the distance matrix so that the maximum distance is 1
    max_distance = np.sqrt(center_x**2 + center_y**2)
    normalized_distance = distance_from_center / max_distance

    # Create a weight matrix where weights linearly decrease from 1 at the center to 0 at the edges
    weight_matrix = 1 - normalized_distance

    # Normalization for overlaps was incorrect in the initial approach, let's correct it:
    # The matrix already ensures the center has weight 1.
    # To accommodate overlapping sum to 1, no further division is required assuming
    # that the sum of any two overlapping areas naturally scales with this setup.

    return weight_matrix

# Example usage:
height, width = 256, 256  # Adjust as needed
weight_matrix = create_weight_matrix(height, width)

In [14]:
# this will loop through all files from the input path and make tif files for output path and wheighted path.
# List all files in the directory
for filename in os.listdir(input_01_path):
    # Check for raster files, assuming they are TIFFs
    if filename.endswith('.tif'):
        file_path = os.path.join(input_01_path, filename)
        output_path_file = os.path.join(output_01_path, f"output_{filename}")  # Name output files uniquely
        weighted_path_file = os.path.join(output_weight_01_path, f"weighted_{filename}")  # Name weighted files uniquely

        # Open the file using the correct path
        with rasterio.open(file_path) as src:
            out_meta = src.meta.copy()
            out_meta.update({
                'dtype': 'float32',  # Corrected the typo 'dytpe' to 'dtype'
                'count': 7  # Assuming you want to output 7 bands
            })

            # Assuming haleiwa_01_results is previously defined and holds your model outputs
            # data_out should be shaped (bands, height, width), thus transposing might be needed
            data_out = haleiwa_01_prediction[0]  # Example placeholder, adjust as necessary
            data_out = np.moveaxis(data_out, -1, 0)  # Rearrange dimensions if necessary
            weight_matrix = create_weight_matrix(384,384)
            weighted_data_out = weight_matrix*data_out
            with rasterio.open(output_path_file, 'w', **out_meta) as dst:
              dst.write(data_out)
              with rasterio.open(weighted_path_file, 'w', **out_meta) as dst:
                dst.write(weighted_data_out)

In [15]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import matplotlib.pyplot as plt

def create_mosaic_sum(input_rasters):
    # Open all rasters
    src_files = [rasterio.open(path) for path in input_rasters]

    # Use rasterio's merge function with method='sum' to sum overlapping pixels
    mosaic, out_trans = merge(src_files, method='sum')

    # Define the metadata for the output GeoTIFF
    out_meta = src_files[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "count": src_files[0].count
    })
       # Write the mosaic raster to disk
    with rasterio.open('haleiwa_01_mosaic_sum.tif', 'w', **out_meta) as dest:
        dest.write(mosaic)

    # Close all raster files
    for src in src_files:
        src.close()

    return mosaic, out_trans


In [16]:
input_rasters = glob.glob("/content/haleiwa_01_output_weight/*.tif")

In [ ]:
input_rasters

In [18]:
mosaic, out_trans = create_mosaic_sum(input_rasters)

In [19]:
mosaic.shape

(7, 5760, 4608)

In [ ]:
import rasterio
import matplotlib.pyplot as plt

def view_mosaic(file_path, band=1):
    """View a specified band of a mosaic using matplotlib."""
    with rasterio.open(file_path) as src:
        # Read a specific band (default to the first band)
        data = src.read(band)

        # Plot the data
        plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
        plt.imshow(data, cmap='viridis')  # Choose a colormap that fits your data
        plt.colorbar(label='Digital number values')
        plt.title('Mosaic Visualization')
        plt.xlabel('Column #')
        plt.ylabel('Row #')
        plt.show()

# Path to the mosaic TIFF file
mosaic_file_path = '/content/haleiwa_01_mosaic_sum.tif'

# Call the function to view the mosaic
view_mosaic(mosaic_file_path)


In [295]:

import zipfile
import os

# Path to the file you want to zip
file_to_zip = '/content/haleiwa_03_mosaic.tif'
# Path to the folder where you want to save the zip file
output_folder = '/content/drive/MyDrive/data/model_prediction/probability_data'
# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)
# The name of the resulting zip file, including the path to the output folder
zip_file_name = os.path.join(output_folder, 'haleiwa_03_mosaic.zip')

# Creating a zip file in write mode
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Adding the file to the zip file
    zipf.write(file_to_zip, arcname=os.path.basename(file_to_zip))

print(f'File {file_to_zip} has been zipped as {zip_file_name}')

# Path to the file you want to zip
file_to_zip = '/content/kahuku_01_mosaic.tif'
# Path to the folder where you want to save the zip file
output_folder = '/content/drive/MyDrive/data/model_prediction/probability_data'
# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)
# The name of the resulting zip file, including the path to the output folder
zip_file_name = os.path.join(output_folder, 'kahuku_01_mosaic.zip')

# Creating a zip file in write mode
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Adding the file to the zip file
    zipf.write(file_to_zip, arcname=os.path.basename(file_to_zip))

print(f'File {file_to_zip} has been zipped as {zip_file_name}')


# Path to the file you want to zip
file_to_zip = '/content/kahuku_02_mosaic.tif'
# Path to the folder where you want to save the zip file
output_folder = '/content/drive/MyDrive/data/model_prediction/probability_data'
# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)
# The name of the resulting zip file, including the path to the output folder
zip_file_name = os.path.join(output_folder, 'kahuku_02_mosaic.zip')

# Creating a zip file in write mode
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Adding the file to the zip file
    zipf.write(file_to_zip, arcname=os.path.basename(file_to_zip))

print(f'File {file_to_zip} has been zipped as {zip_file_name}')


# Path to the file you want to zip
file_to_zip = '/content/kahuku_03_mosaic.tif'
# Path to the folder where you want to save the zip file
output_folder = '/content/drive/MyDrive/data/model_prediction/probability_data'
# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)
# The name of the resulting zip file, including the path to the output folder
zip_file_name = os.path.join(output_folder, 'kahuku_03_mosaic.zip')

# Creating a zip file in write mode
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Adding the file to the zip file
    zipf.write(file_to_zip, arcname=os.path.basename(file_to_zip))

print(f'File {file_to_zip} has been zipped as {zip_file_name}')


# Path to the file you want to zip
file_to_zip = '/content/kawainui_01_mosaic.tif'
# Path to the folder where you want to save the zip file
output_folder = '/content/drive/MyDrive/data/model_prediction/probability_data'
# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)
# The name of the resulting zip file, including the path to the output folder
zip_file_name = os.path.join(output_folder, 'kawainui_01_mosaic.zip')

# Creating a zip file in write mode
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Adding the file to the zip file
    zipf.write(file_to_zip, arcname=os.path.basename(file_to_zip))

print(f'File {file_to_zip} has been zipped as {zip_file_name}')

# Path to the file you want to zip
file_to_zip = '/content/kawainui_02_mosaic.tif'
# Path to the folder where you want to save the zip file
output_folder = '/content/drive/MyDrive/data/model_prediction/probability_data'
# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)
# The name of the resulting zip file, including the path to the output folder
zip_file_name = os.path.join(output_folder, 'kawainui_02_mosaic.zip')

# Creating a zip file in write mode
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Adding the file to the zip file
    zipf.write(file_to_zip, arcname=os.path.basename(file_to_zip))

print(f'File {file_to_zip} has been zipped as {zip_file_name}')

# Path to the file you want to zip
file_to_zip = '/content/kawainui_03_mosaic.tif'
# Path to the folder where you want to save the zip file
output_folder = '/content/drive/MyDrive/data/model_prediction/probability_data'
# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)
# The name of the resulting zip file, including the path to the output folder
zip_file_name = os.path.join(output_folder, 'kawainui_03_mosaic.zip')

# Creating a zip file in write mode
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Adding the file to the zip file
    zipf.write(file_to_zip, arcname=os.path.basename(file_to_zip))

print(f'File {file_to_zip} has been zipped as {zip_file_name}')



File /content/haleiwa_03_mosaic.tif has been zipped as /content/drive/MyDrive/data/model_prediction/probability_data/haleiwa_03_mosaic.zip
File /content/kahuku_01_mosaic.tif has been zipped as /content/drive/MyDrive/data/model_prediction/probability_data/kahuku_01_mosaic.zip
File /content/kahuku_02_mosaic.tif has been zipped as /content/drive/MyDrive/data/model_prediction/probability_data/kahuku_02_mosaic.zip
File /content/kahuku_03_mosaic.tif has been zipped as /content/drive/MyDrive/data/model_prediction/probability_data/kahuku_03_mosaic.zip
File /content/kawainui_01_mosaic.tif has been zipped as /content/drive/MyDrive/data/model_prediction/probability_data/kawainui_01_mosaic.zip
File /content/kawainui_02_mosaic.tif has been zipped as /content/drive/MyDrive/data/model_prediction/probability_data/kawainui_02_mosaic.zip
File /content/kawainui_03_mosaic.tif has been zipped as /content/drive/MyDrive/data/model_prediction/probability_data/kawainui_03_mosaic.zip


In [21]:
zip_name = 'haleiwa_01_output'
directory_name = '/content/haleiwa_01_output'

# Create 'path\to\zip_file.zip'
shutil.make_archive(zip_name, 'zip', directory_name)

'/content/haleiwa_01_output.zip'

In [22]:
zip_name = 'haleiwa_01'
directory_name = '/content/output_weight_01'

# Create 'path\to\zip_file.zip'
shutil.make_archive(zip_name, 'zip', directory_name)

'/content/haleiwa_01_output_weight.zip'

time to do it one at a time

In [167]:
# for 1 year images - this with handling of band 5
def read_thirteen_band_image(file_path):
    with rasterio.open(file_path) as src:
      bands = [src.read(i) for i in range(1,13)]

      image = np.stack(bands, axis=-1)

      image = image.astype(float)
      max_value = image.max(axis=(0,1), keepdims=True)
      min_value = image.min(axis=(0,1), keepdims=True)
      image = (image - min_value) / (max_value - min_value)

      band_five = src.read(13)

      band_five = band_five.astype(float) /band_five.max()
      band_five = np.expand_dims(band_five, axis=-1)
      combined_image = np.concatenate([image, band_five], axis=-1)


      return combined_image

In [107]:
# for 1 year images - this with handling of band 9
def read_nine_band_image(file_path):
    with rasterio.open(file_path) as src:
      bands = [src.read(i) for i in range(1,9)]

      image = np.stack(bands, axis=-1)

      image = image.astype(float)
      max_value = image.max(axis=(0,1), keepdims=True)
      min_value = image.min(axis=(0,1), keepdims=True)
      image = (image - min_value) / (max_value - min_value)

      band_five = src.read(9)

      band_five = band_five.astype(float) /band_five.max()
      band_five = np.expand_dims(band_five, axis=-1)
      combined_image = np.concatenate([image, band_five], axis=-1)


      return combined_image

In [267]:
tile_1465 = read_thirteen_band_image("/content/kawainui_3year/1465_3year.tif")
tile_1466 = read_thirteen_band_image("/content/kawainui_3year/1466_3year.tif")
tile_1472 = read_thirteen_band_image("/content/kawainui_3year/1472_3year.tif")
tile_1481 = read_thirteen_band_image("/content/kawainui_3year/1481_3year.tif")
tile_1482 = read_thirteen_band_image("/content/kawainui_3year/1482_3year.tif")
tile_1483 = read_thirteen_band_image("/content/kawainui_3year/1483_3year.tif")
tile_1488 = read_thirteen_band_image("/content/kawainui_3year/1488_3year.tif")
tile_1489 = read_thirteen_band_image("/content/kawainui_3year/1489_3year.tif")
tile_1490 = read_thirteen_band_image("/content/kawainui_3year/1490_3year.tif")
tile_1497 = read_thirteen_band_image("/content/kawainui_3year/1497_3year.tif")
tile_1498 = read_thirteen_band_image("/content/kawainui_3year/1498_3year.tif")
tile_1499 = read_thirteen_band_image("/content/kawainui_3year/1499_3year.tif")
tile_1504 = read_thirteen_band_image("/content/kawainui_3year/1504_3year.tif")
tile_1505 = read_thirteen_band_image("/content/kawainui_3year/1505_3year.tif")
tile_1507 = read_thirteen_band_image("/content/kawainui_3year/1507_3year.tif")
tile_1513 = read_thirteen_band_image("/content/kawainui_3year/1513_3year.tif")
tile_1514 = read_thirteen_band_image("/content/kawainui_3year/1514_3year.tif")
tile_1515 = read_thirteen_band_image("/content/kawainui_3year/1515_3year.tif")
tile_1516 = read_thirteen_band_image("/content/kawainui_3year/1516_3year.tif")
tile_1517 = read_thirteen_band_image("/content/kawainui_3year/1517_3year.tif")
tile_1520 = read_thirteen_band_image("/content/kawainui_3year/1520_3year.tif")
tile_1521 = read_thirteen_band_image("/content/kawainui_3year/1521_3year.tif")
tile_1529 = read_thirteen_band_image("/content/kawainui_3year/1529_3year.tif")
tile_1530 = read_thirteen_band_image("/content/kawainui_3year/1530_3year.tif")
tile_1531 = read_thirteen_band_image("/content/kawainui_3year/1531_3year.tif")
tile_1532 = read_thirteen_band_image("/content/kawainui_3year/1532_3year.tif")
tile_1533 = read_thirteen_band_image("/content/kawainui_3year/1533_3year.tif")
tile_1534 = read_thirteen_band_image("/content/kawainui_3year/1534_3year.tif")
tile_1535 = read_thirteen_band_image("/content/kawainui_3year/1535_3year.tif")
tile_1536 = read_thirteen_band_image("/content/kawainui_3year/1536_3year.tif")
tile_1537 = read_thirteen_band_image("/content/kawainui_3year/1537_3year.tif")
tile_1538 = read_thirteen_band_image("/content/kawainui_3year/1538_3year.tif")
tile_1539 = read_thirteen_band_image("/content/kawainui_3year/1539_3year.tif")
tile_1545 = read_thirteen_band_image("/content/kawainui_3year/1545_3year.tif")
tile_1546 = read_thirteen_band_image("/content/kawainui_3year/1546_3year.tif")
tile_1547 = read_thirteen_band_image("/content/kawainui_3year/1547_3year.tif")
tile_1548 = read_thirteen_band_image("/content/kawainui_3year/1548_3year.tif")
tile_1549 = read_thirteen_band_image("/content/kawainui_3year/1549_3year.tif")
tile_1550 = read_thirteen_band_image("/content/kawainui_3year/1550_3year.tif")
tile_1551 = read_thirteen_band_image("/content/kawainui_3year/1551_3year.tif")
tile_1555 = read_thirteen_band_image("/content/kawainui_3year/1555_3year.tif")
tile_1561 = read_thirteen_band_image("/content/kawainui_3year/1561_3year.tif")
tile_1562 = read_thirteen_band_image("/content/kawainui_3year/1562_3year.tif")
tile_1563 = read_thirteen_band_image("/content/kawainui_3year/1563_3year.tif")
tile_1564 = read_thirteen_band_image("/content/kawainui_3year/1564_3year.tif")
tile_1565 = read_thirteen_band_image("/content/kawainui_3year/1565_3year.tif")
tile_1571 = read_thirteen_band_image("/content/kawainui_3year/1571_3year.tif")
tile_1577 = read_thirteen_band_image("/content/kawainui_3year/1577_3year.tif")
tile_1578 = read_thirteen_band_image("/content/kawainui_3year/1578_3year.tif")
tile_1579 = read_thirteen_band_image("/content/kawainui_3year/1579_3year.tif")
tile_1580 = read_thirteen_band_image("/content/kawainui_3year/1580_3year.tif")
tile_1593 = read_thirteen_band_image("/content/kawainui_3year/1593_3year.tif")
tile_1594 = read_thirteen_band_image("/content/kawainui_3year/1594_3year.tif")
tile_1595 = read_thirteen_band_image("/content/kawainui_3year/1595_3year.tif")
tile_1596 = read_thirteen_band_image("/content/kawainui_3year/1596_3year.tif")
tile_1610 = read_thirteen_band_image("/content/kawainui_3year/1610_3year.tif")
tile_1611 = read_thirteen_band_image("/content/kawainui_3year/1611_3year.tif")
tile_1612 = read_thirteen_band_image("/content/kawainui_3year/1612_3year.tif")
tile_1628 = read_thirteen_band_image("/content/kawainui_3year/1628_3year.tif")
tile_1645 = read_thirteen_band_image("/content/kawainui_3year/1645_3year.tif")

In [258]:
# tile_1465 = read_nine_band_image("/content/kawainui_2year/1465_2year.tif")

# tile_1466 = read_nine_band_image("/content/kawainui_2year/1466_2year.tif")
# tile_1472 = read_nine_band_image("/content/kawainui_2year/1472_2year.tif")
# tile_1481 = read_nine_band_image("/content/kawainui_2year/1481_2year.tif")
# tile_1482 = read_nine_band_image("/content/kawainui_2year/1482_2year.tif")
# tile_1483 = read_nine_band_image("/content/kawainui_2year/1483_2year.tif")
# tile_1488 = read_nine_band_image("/content/kawainui_2year/1488_2year.tif")
# tile_1489 = read_nine_band_image("/content/kawainui_2year/1489_2year.tif")
# tile_1490 = read_nine_band_image("/content/kawainui_2year/1490_2year.tif")
# tile_1497 = read_nine_band_image("/content/kawainui_2year/1497_2year.tif")
# tile_1498 = read_nine_band_image("/content/kawainui_2year/1498_2year.tif")
# tile_1499 = read_nine_band_image("/content/kawainui_2year/1499_2year.tif")
# tile_1504 = read_nine_band_image("/content/kawainui_2year/1504_2year.tif")
# tile_1505 = read_nine_band_image("/content/kawainui_2year/1505_2year.tif")
# tile_1507 = read_nine_band_image("/content/kawainui_2year/1507_2year.tif")
# tile_1513 = read_nine_band_image("/content/kawainui_2year/1513_2year.tif")
# tile_1514 = read_nine_band_image("/content/kawainui_2year/1514_2year.tif")
# tile_1515 = read_nine_band_image("/content/kawainui_2year/1515_2year.tif")
# tile_1516 = read_nine_band_image("/content/kawainui_2year/1516_2year.tif")
# tile_1517 = read_nine_band_image("/content/kawainui_2year/1517_2year.tif")
# tile_1520 = read_nine_band_image("/content/kawainui_2year/1520_2year.tif")
# tile_1521 = read_nine_band_image("/content/kawainui_2year/1521_2year.tif")
# tile_1529 = read_nine_band_image("/content/kawainui_2year/1529_2year.tif")
# tile_1530 = read_nine_band_image("/content/kawainui_2year/1530_2year.tif")
# tile_1531 = read_nine_band_image("/content/kawainui_2year/1531_2year.tif")
# tile_1532 = read_nine_band_image("/content/kawainui_2year/1532_2year.tif")
# tile_1533 = read_nine_band_image("/content/kawainui_2year/1533_2year.tif")
# tile_1534 = read_nine_band_image("/content/kawainui_2year/1534_2year.tif")
# tile_1535 = read_nine_band_image("/content/kawainui_2year/1535_2year.tif")
# tile_1536 = read_nine_band_image("/content/kawainui_2year/1536_2year.tif")
# tile_1537 = read_nine_band_image("/content/kawainui_2year/1537_2year.tif")
# tile_1538 = read_nine_band_image("/content/kawainui_2year/1538_2year.tif")
# tile_1539 = read_nine_band_image("/content/kawainui_2year/1539_2year.tif")
# tile_1545 = read_nine_band_image("/content/kawainui_2year/1545_2year.tif")
# tile_1546 = read_nine_band_image("/content/kawainui_2year/1546_2year.tif")
# tile_1547 = read_nine_band_image("/content/kawainui_2year/1547_2year.tif")
# tile_1548 = read_nine_band_image("/content/kawainui_2year/1548_2year.tif")
# tile_1549 = read_nine_band_image("/content/kawainui_2year/1549_2year.tif")
# tile_1550 = read_nine_band_image("/content/kawainui_2year/1550_2year.tif")
# tile_1551 = read_nine_band_image("/content/kawainui_2year/1551_2year.tif")
# tile_1555 = read_nine_band_image("/content/kawainui_2year/1555_2year.tif")
# tile_1561 = read_nine_band_image("/content/kawainui_2year/1561_2year.tif")
# tile_1562 = read_nine_band_image("/content/kawainui_2year/1562_2year.tif")
# tile_1563 = read_nine_band_image("/content/kawainui_2year/1563_2year.tif")
# tile_1564 = read_nine_band_image("/content/kawainui_2year/1564_2year.tif")
# tile_1565 = read_nine_band_image("/content/kawainui_2year/1565_2year.tif")
# tile_1571 = read_nine_band_image("/content/kawainui_2year/1571_2year.tif")
# tile_1577 = read_nine_band_image("/content/kawainui_2year/1577_2year.tif")
# tile_1578 = read_nine_band_image("/content/kawainui_2year/1578_2year.tif")
# tile_1579 = read_nine_band_image("/content/kawainui_2year/1579_2year.tif")
# tile_1580 = read_nine_band_image("/content/kawainui_2year/1580_2year.tif")
# tile_1593 = read_nine_band_image("/content/kawainui_2year/1593_2year.tif")
# tile_1594 = read_nine_band_image("/content/kawainui_2year/1594_2year.tif")
# tile_1595 = read_nine_band_image("/content/kawainui_2year/1595_2year.tif")
# tile_1596 = read_nine_band_image("/content/kawainui_2year/1596_2year.tif")
# tile_1610 = read_nine_band_image("/content/kawainui_2year/1610_2year.tif")
# tile_1611 = read_nine_band_image("/content/kawainui_2year/1611_2year.tif")
# tile_1612 = read_nine_band_image("/content/kawainui_2year/1612_2year.tif")
# tile_1628 = read_nine_band_image("/content/kawainui_2year/1628_2year.tif")
# tile_1645 = read_nine_band_image("/content/kawainui_2year/1645_2year.tif")



In [251]:
# tile_1465 = read_multi_image_withfive("/content/kawainui_1year/1465_1year.tif")
# tile_1466 = read_multi_image_withfive("/content/kawainui_1year/1466_1year.tif")
# tile_1472 = read_multi_image_withfive("/content/kawainui_1year/1472_1year.tif")
# tile_1481 = read_multi_image_withfive("/content/kawainui_1year/1481_1year.tif")
# tile_1482 = read_multi_image_withfive("/content/kawainui_1year/1482_1year.tif")
# tile_1483 = read_multi_image_withfive("/content/kawainui_1year/1483_1year.tif")
# tile_1488 = read_multi_image_withfive("/content/kawainui_1year/1488_1year.tif")
# tile_1489 = read_multi_image_withfive("/content/kawainui_1year/1489_1year.tif")
# tile_1490 = read_multi_image_withfive("/content/kawainui_1year/1490_1year.tif")
# tile_1497 = read_multi_image_withfive("/content/kawainui_1year/1497_1year.tif")
# tile_1498 = read_multi_image_withfive("/content/kawainui_1year/1498_1year.tif")
# tile_1499 = read_multi_image_withfive("/content/kawainui_1year/1499_1year.tif")
# tile_1504 = read_multi_image_withfive("/content/kawainui_1year/1504_1year.tif")
# tile_1505 = read_multi_image_withfive("/content/kawainui_1year/1505_1year.tif")
# tile_1507 = read_multi_image_withfive("/content/kawainui_1year/1507_1year.tif")
# tile_1513 = read_multi_image_withfive("/content/kawainui_1year/1513_1year.tif")
# tile_1514 = read_multi_image_withfive("/content/kawainui_1year/1514_1year.tif")
# tile_1515 = read_multi_image_withfive("/content/kawainui_1year/1515_1year.tif")
# tile_1516 = read_multi_image_withfive("/content/kawainui_1year/1516_1year.tif")
# tile_1517 = read_multi_image_withfive("/content/kawainui_1year/1517_1year.tif")
# tile_1520 = read_multi_image_withfive("/content/kawainui_1year/1520_1year.tif")
# tile_1521 = read_multi_image_withfive("/content/kawainui_1year/1521_1year.tif")
# tile_1529 = read_multi_image_withfive("/content/kawainui_1year/1529_1year.tif")
# tile_1530 = read_multi_image_withfive("/content/kawainui_1year/1530_1year.tif")
# tile_1531 = read_multi_image_withfive("/content/kawainui_1year/1531_1year.tif")
# tile_1532 = read_multi_image_withfive("/content/kawainui_1year/1532_1year.tif")
# tile_1533 = read_multi_image_withfive("/content/kawainui_1year/1533_1year.tif")
# tile_1534 = read_multi_image_withfive("/content/kawainui_1year/1534_1year.tif")
# tile_1535 = read_multi_image_withfive("/content/kawainui_1year/1535_1year.tif")
# tile_1536 = read_multi_image_withfive("/content/kawainui_1year/1536_1year.tif")
# tile_1537 = read_multi_image_withfive("/content/kawainui_1year/1537_1year.tif")
# tile_1538 = read_multi_image_withfive("/content/kawainui_1year/1538_1year.tif")
# tile_1539 = read_multi_image_withfive("/content/kawainui_1year/1539_1year.tif")
# tile_1545 = read_multi_image_withfive("/content/kawainui_1year/1545_1year.tif")
# tile_1546 = read_multi_image_withfive("/content/kawainui_1year/1546_1year.tif")
# tile_1547 = read_multi_image_withfive("/content/kawainui_1year/1547_1year.tif")
# tile_1548 = read_multi_image_withfive("/content/kawainui_1year/1548_1year.tif")
# tile_1549 = read_multi_image_withfive("/content/kawainui_1year/1549_1year.tif")
# tile_1550 = read_multi_image_withfive("/content/kawainui_1year/1550_1year.tif")
# tile_1551 = read_multi_image_withfive("/content/kawainui_1year/1551_1year.tif")
# tile_1555 = read_multi_image_withfive("/content/kawainui_1year/1555_1year.tif")
# tile_1561 = read_multi_image_withfive("/content/kawainui_1year/1561_1year.tif")
# tile_1562 = read_multi_image_withfive("/content/kawainui_1year/1562_1year.tif")
# tile_1563 = read_multi_image_withfive("/content/kawainui_1year/1563_1year.tif")
# tile_1564 = read_multi_image_withfive("/content/kawainui_1year/1564_1year.tif")
# tile_1565 = read_multi_image_withfive("/content/kawainui_1year/1565_1year.tif")
# tile_1571 = read_multi_image_withfive("/content/kawainui_1year/1571_1year.tif")
# tile_1577 = read_multi_image_withfive("/content/kawainui_1year/1577_1year.tif")
# tile_1578 = read_multi_image_withfive("/content/kawainui_1year/1578_1year.tif")
# tile_1579 = read_multi_image_withfive("/content/kawainui_1year/1579_1year.tif")
# tile_1580 = read_multi_image_withfive("/content/kawainui_1year/1580_1year.tif")
# tile_1593 = read_multi_image_withfive("/content/kawainui_1year/1593_1year.tif")
# tile_1594 = read_multi_image_withfive("/content/kawainui_1year/1594_1year.tif")
# tile_1595 = read_multi_image_withfive("/content/kawainui_1year/1595_1year.tif")
# tile_1596 = read_multi_image_withfive("/content/kawainui_1year/1596_1year.tif")
# tile_1610 = read_multi_image_withfive("/content/kawainui_1year/1610_1year.tif")
# tile_1611 = read_multi_image_withfive("/content/kawainui_1year/1611_1year.tif")
# tile_1612 = read_multi_image_withfive("/content/kawainui_1year/1612_1year.tif")
# tile_1628 = read_multi_image_withfive("/content/kawainui_1year/1628_1year.tif")
# tile_1645 = read_multi_image_withfive("/content/kawainui_1year/1645_1year.tif")

In [268]:
tile_1465 = np.expand_dims(tile_1465, axis=0)
tile_1466 = np.expand_dims(tile_1466, axis=0)
tile_1472 = np.expand_dims(tile_1472, axis=0)
tile_1481 = np.expand_dims(tile_1481, axis=0)
tile_1482 = np.expand_dims(tile_1482, axis=0)
tile_1483 = np.expand_dims(tile_1483, axis=0)
tile_1488 = np.expand_dims(tile_1488, axis=0)
tile_1489 = np.expand_dims(tile_1489, axis=0)
tile_1490 = np.expand_dims(tile_1490, axis=0)
tile_1497 = np.expand_dims(tile_1497, axis=0)
tile_1498 = np.expand_dims(tile_1498, axis=0)
tile_1499 = np.expand_dims(tile_1499, axis=0)
tile_1504 = np.expand_dims(tile_1504, axis=0)
tile_1505 = np.expand_dims(tile_1505, axis=0)
tile_1507 = np.expand_dims(tile_1507, axis=0)
tile_1513 = np.expand_dims(tile_1513, axis=0)
tile_1514 = np.expand_dims(tile_1514, axis=0)
tile_1515 = np.expand_dims(tile_1515, axis=0)
tile_1516 = np.expand_dims(tile_1516, axis=0)
tile_1517 = np.expand_dims(tile_1517, axis=0)
tile_1520 = np.expand_dims(tile_1520, axis=0)
tile_1521 = np.expand_dims(tile_1521, axis=0)
tile_1529 = np.expand_dims(tile_1529, axis=0)
tile_1530 = np.expand_dims(tile_1530, axis=0)
tile_1531 = np.expand_dims(tile_1531, axis=0)
tile_1532 = np.expand_dims(tile_1532, axis=0)
tile_1533 = np.expand_dims(tile_1533, axis=0)
tile_1534 = np.expand_dims(tile_1534, axis=0)
tile_1535 = np.expand_dims(tile_1535, axis=0)
tile_1536 = np.expand_dims(tile_1536, axis=0)
tile_1537 = np.expand_dims(tile_1537, axis=0)
tile_1538 = np.expand_dims(tile_1538, axis=0)
tile_1539 = np.expand_dims(tile_1539, axis=0)
tile_1545 = np.expand_dims(tile_1545, axis=0)
tile_1546 = np.expand_dims(tile_1546, axis=0)
tile_1547 = np.expand_dims(tile_1547, axis=0)
tile_1548 = np.expand_dims(tile_1548, axis=0)
tile_1549 = np.expand_dims(tile_1549, axis=0)
tile_1550 = np.expand_dims(tile_1550, axis=0)
tile_1551 = np.expand_dims(tile_1551, axis=0)
tile_1555 = np.expand_dims(tile_1555, axis=0)
tile_1561 = np.expand_dims(tile_1561, axis=0)
tile_1562 = np.expand_dims(tile_1562, axis=0)
tile_1563 = np.expand_dims(tile_1563, axis=0)
tile_1564 = np.expand_dims(tile_1564, axis=0)
tile_1565 = np.expand_dims(tile_1565, axis=0)
tile_1571 = np.expand_dims(tile_1571, axis=0)
tile_1577 = np.expand_dims(tile_1577, axis=0)
tile_1578 = np.expand_dims(tile_1578, axis=0)
tile_1579 = np.expand_dims(tile_1579, axis=0)
tile_1580 = np.expand_dims(tile_1580, axis=0)
tile_1593 = np.expand_dims(tile_1593, axis=0)
tile_1594 = np.expand_dims(tile_1594, axis=0)
tile_1595 = np.expand_dims(tile_1595, axis=0)
tile_1596 = np.expand_dims(tile_1596, axis=0)
tile_1610 = np.expand_dims(tile_1610, axis=0)
tile_1611 = np.expand_dims(tile_1611, axis=0)
tile_1612 = np.expand_dims(tile_1612, axis=0)
tile_1628 = np.expand_dims(tile_1628, axis=0)
tile_1645 = np.expand_dims(tile_1645, axis=0)

In [245]:
# # 5band kahuku
# tile_742 = read_multi_image_withfive("/content/kahuku_1year/742_1year.tif")
# tile_743 = read_multi_image_withfive("/content/kahuku_1year/743_1year.tif")
# tile_759 = read_multi_image_withfive("/content/kahuku_1year/759_1year.tif")
# tile_760 = read_multi_image_withfive("/content/kahuku_1year/760_1year.tif")
# tile_761 = read_multi_image_withfive("/content/kahuku_1year/761_1year.tif")
# tile_762 = read_multi_image_withfive("/content/kahuku_1year/762_1year.tif")
# tile_776 = read_multi_image_withfive("/content/kahuku_1year/776_1year.tif")
# tile_777 = read_multi_image_withfive("/content/kahuku_1year/777_1year.tif")
# tile_778 = read_multi_image_withfive("/content/kahuku_1year/778_1year.tif")
# tile_779 = read_multi_image_withfive("/content/kahuku_1year/779_1year.tif")
# tile_780 = read_multi_image_withfive("/content/kahuku_1year/780_1year.tif")
# tile_781 = read_multi_image_withfive("/content/kahuku_1year/781_1year.tif")
# tile_792 = read_multi_image_withfive("/content/kahuku_1year/792_1year.tif")
# tile_793 = read_multi_image_withfive("/content/kahuku_1year/793_1year.tif")
# tile_794 = read_multi_image_withfive("/content/kahuku_1year/794_1year.tif")
# tile_795 = read_multi_image_withfive("/content/kahuku_1year/795_1year.tif")
# tile_796 = read_multi_image_withfive("/content/kahuku_1year/796_1year.tif")
# tile_797 = read_multi_image_withfive("/content/kahuku_1year/797_1year.tif")
# tile_798 = read_multi_image_withfive("/content/kahuku_1year/798_1year.tif")
# tile_805 = read_multi_image_withfive("/content/kahuku_1year/805_1year.tif")
# tile_808 = read_multi_image_withfive("/content/kahuku_1year/808_1year.tif")
# tile_809 = read_multi_image_withfive("/content/kahuku_1year/809_1year.tif")
# tile_810 = read_multi_image_withfive("/content/kahuku_1year/810_1year.tif")
# tile_811 = read_multi_image_withfive("/content/kahuku_1year/811_1year.tif")
# tile_812 = read_multi_image_withfive("/content/kahuku_1year/812_1year.tif")
# tile_813 = read_multi_image_withfive("/content/kahuku_1year/813_1year.tif")
# tile_815 = read_multi_image_withfive("/content/kahuku_1year/815_1year.tif")
# tile_821 = read_multi_image_withfive("/content/kahuku_1year/821_1year.tif")
# tile_823 = read_multi_image_withfive("/content/kahuku_1year/823_1year.tif")
# tile_825 = read_multi_image_withfive("/content/kahuku_1year/825_1year.tif")
# tile_826 = read_multi_image_withfive("/content/kahuku_1year/826_1year.tif")
# tile_827 = read_multi_image_withfive("/content/kahuku_1year/827_1year.tif")
# tile_828 = read_multi_image_withfive("/content/kahuku_1year/828_1year.tif")
# tile_829 = read_multi_image_withfive("/content/kahuku_1year/829_1year.tif")
# tile_830 = read_multi_image_withfive("/content/kahuku_1year/830_1year.tif")
# tile_831 = read_multi_image_withfive("/content/kahuku_1year/831_1year.tif")
# tile_841 = read_multi_image_withfive("/content/kahuku_1year/841_1year.tif")
# tile_842 = read_multi_image_withfive("/content/kahuku_1year/842_1year.tif")
# tile_843 = read_multi_image_withfive("/content/kahuku_1year/843_1year.tif")
# tile_845 = read_multi_image_withfive("/content/kahuku_1year/845_1year.tif")
# tile_847 = read_multi_image_withfive("/content/kahuku_1year/847_1year.tif")
# tile_848 = read_multi_image_withfive("/content/kahuku_1year/848_1year.tif")
# tile_849 = read_multi_image_withfive("/content/kahuku_1year/849_1year.tif")
# tile_860 = read_multi_image_withfive("/content/kahuku_1year/860_1year.tif")
# tile_861 = read_multi_image_withfive("/content/kahuku_1year/861_1year.tif")
# tile_862 = read_multi_image_withfive("/content/kahuku_1year/862_1year.tif")
# tile_865 = read_multi_image_withfive("/content/kahuku_1year/865_1year.tif")
# tile_866 = read_multi_image_withfive("/content/kahuku_1year/866_1year.tif")

In [236]:
# tile_742 = read_nine_band_image("/content/kahuku_2year/742_2year.tif")
# tile_743 = read_nine_band_image("/content/kahuku_2year/743_2year.tif")
# tile_759 = read_nine_band_image("/content/kahuku_2year/759_2year.tif")
# tile_760 = read_nine_band_image("/content/kahuku_2year/760_2year.tif")
# tile_761 = read_nine_band_image("/content/kahuku_2year/761_2year.tif")
# tile_762 = read_nine_band_image("/content/kahuku_2year/762_2year.tif")
# tile_776 = read_nine_band_image("/content/kahuku_2year/776_2year.tif")
# tile_777 = read_nine_band_image("/content/kahuku_2year/777_2year.tif")
# tile_778 = read_nine_band_image("/content/kahuku_2year/778_2year.tif")
# tile_779 = read_nine_band_image("/content/kahuku_2year/779_2year.tif")
# tile_780 = read_nine_band_image("/content/kahuku_2year/780_2year.tif")
# tile_781 = read_nine_band_image("/content/kahuku_2year/781_2year.tif")
# tile_792 = read_nine_band_image("/content/kahuku_2year/792_2year.tif")
# tile_793 = read_nine_band_image("/content/kahuku_2year/793_2year.tif")
# tile_794 = read_nine_band_image("/content/kahuku_2year/794_2year.tif")
# tile_795 = read_nine_band_image("/content/kahuku_2year/795_2year.tif")
# tile_796 = read_nine_band_image("/content/kahuku_2year/796_2year.tif")
# tile_797 = read_nine_band_image("/content/kahuku_2year/797_2year.tif")
# tile_798 = read_nine_band_image("/content/kahuku_2year/798_2year.tif")
# tile_805 = read_nine_band_image("/content/kahuku_2year/805_2year.tif")
# tile_808 = read_nine_band_image("/content/kahuku_2year/808_2year.tif")
# tile_809 = read_nine_band_image("/content/kahuku_2year/809_2year.tif")
# tile_810 = read_nine_band_image("/content/kahuku_2year/810_2year.tif")
# tile_811 = read_nine_band_image("/content/kahuku_2year/811_2year.tif")
# tile_812 = read_nine_band_image("/content/kahuku_2year/812_2year.tif")
# tile_813 = read_nine_band_image("/content/kahuku_2year/813_2year.tif")
# tile_815 = read_nine_band_image("/content/kahuku_2year/815_2year.tif")
# tile_821 = read_nine_band_image("/content/kahuku_2year/821_2year.tif")
# tile_823 = read_nine_band_image("/content/kahuku_2year/823_2year.tif")
# tile_825 = read_nine_band_image("/content/kahuku_2year/825_2year.tif")
# tile_826 = read_nine_band_image("/content/kahuku_2year/826_2year.tif")
# tile_827 = read_nine_band_image("/content/kahuku_2year/827_2year.tif")
# tile_828 = read_nine_band_image("/content/kahuku_2year/828_2year.tif")
# tile_829 = read_nine_band_image("/content/kahuku_2year/829_2year.tif")
# tile_830 = read_nine_band_image("/content/kahuku_2year/830_2year.tif")
# tile_831 = read_nine_band_image("/content/kahuku_2year/831_2year.tif")
# tile_841 = read_nine_band_image("/content/kahuku_2year/841_2year.tif")
# tile_842 = read_nine_band_image("/content/kahuku_2year/842_2year.tif")
# tile_843 = read_nine_band_image("/content/kahuku_2year/843_2year.tif")
# tile_845 = read_nine_band_image("/content/kahuku_2year/845_2year.tif")
# tile_847 = read_nine_band_image("/content/kahuku_2year/847_2year.tif")
# tile_848 = read_nine_band_image("/content/kahuku_2year/848_2year.tif")
# tile_849 = read_nine_band_image("/content/kahuku_2year/849_2year.tif")
# tile_860 = read_nine_band_image("/content/kahuku_2year/860_2year.tif")
# tile_861 = read_nine_band_image("/content/kahuku_2year/861_2year.tif")
# tile_862 = read_nine_band_image("/content/kahuku_2year/862_2year.tif")
# tile_865 = read_nine_band_image("/content/kahuku_2year/865_2year.tif")
# tile_866 = read_nine_band_image("/content/kahuku_2year/866_2year.tif")



In [246]:
# tile_742 = np.expand_dims(tile_742, axis=0)
# tile_743 = np.expand_dims(tile_743, axis=0)
# tile_759 = np.expand_dims(tile_759, axis=0)
# tile_760 = np.expand_dims(tile_760, axis=0)
# tile_761 = np.expand_dims(tile_761, axis=0)
# tile_762 = np.expand_dims(tile_762, axis=0)
# tile_776 =  np.expand_dims(tile_776, axis=0)
# tile_777 = np.expand_dims(tile_777, axis=0)
# tile_778 =  np.expand_dims(tile_778, axis=0)
# tile_779 = np.expand_dims(tile_779, axis=0)
# tile_780 = np.expand_dims(tile_780, axis=0)
# tile_781 = np.expand_dims(tile_781, axis=0)
# tile_792 = np.expand_dims(tile_792, axis=0)
# tile_793 = np.expand_dims(tile_793, axis=0)
# tile_794 = np.expand_dims(tile_794, axis=0)
# tile_795 = np.expand_dims(tile_795, axis=0)
# tile_796 = np.expand_dims(tile_796, axis=0)
# tile_797 =  np.expand_dims(tile_797, axis=0)
# tile_798 = np.expand_dims(tile_798, axis=0)
# tile_805 =  np.expand_dims(tile_805, axis=0)
# tile_808 = np.expand_dims(tile_808, axis=0)
# tile_809 = np.expand_dims(tile_809, axis=0)
# tile_810 = np.expand_dims(tile_810, axis=0)
# tile_811 = np.expand_dims(tile_811, axis=0)
# tile_812 = np.expand_dims(tile_812, axis=0)
# tile_813 = np.expand_dims(tile_813, axis=0)
# tile_815 = np.expand_dims(tile_815, axis=0)
# tile_821 = np.expand_dims(tile_821, axis=0)
# tile_823 = np.expand_dims(tile_823, axis=0)
# tile_825 = np.expand_dims(tile_825, axis=0)
# tile_826 = np.expand_dims(tile_826, axis=0)
# tile_827 = np.expand_dims(tile_827, axis=0)
# tile_828 = np.expand_dims(tile_828, axis=0)
# tile_829 = np.expand_dims(tile_829, axis=0)
# tile_830 = np.expand_dims(tile_830, axis=0)
# tile_831 = np.expand_dims(tile_831, axis=0)
# tile_841 = np.expand_dims(tile_841, axis=0)
# tile_842 = np.expand_dims(tile_842, axis=0)
# tile_843 = np.expand_dims(tile_843, axis=0)
# tile_845 = np.expand_dims(tile_845, axis=0)
# tile_847 = np.expand_dims(tile_847, axis=0)
# tile_848 = np.expand_dims(tile_848, axis=0)
# tile_849 = np.expand_dims(tile_849, axis=0)
# tile_860 = np.expand_dims(tile_860, axis=0)
# tile_861 = np.expand_dims(tile_861, axis=0)
# tile_862 = np.expand_dims(tile_862, axis=0)
# tile_865 = np.expand_dims(tile_865, axis=0)
# tile_866 = np.expand_dims(tile_866, axis=0)

In [253]:
# tile_1465_result = model_01.predict(tile_1465)
# tile_1466_result = model_01.predict(tile_1466)
# tile_1472_result = model_01.predict(tile_1472)
# tile_1481_result = model_01.predict(tile_1481)
# tile_1482_result = model_01.predict(tile_1482)
# tile_1483_result = model_01.predict(tile_1483)
# tile_1488_result = model_01.predict(tile_1488)
# tile_1489_result = model_01.predict(tile_1489)
# tile_1490_result = model_01.predict(tile_1490)
# tile_1497_result = model_01.predict(tile_1497)
# tile_1498_result = model_01.predict(tile_1498)
# tile_1499_result = model_01.predict(tile_1499)
# tile_1504_result = model_01.predict(tile_1504)
# tile_1505_result = model_01.predict(tile_1505)
# tile_1507_result = model_01.predict(tile_1507)
# tile_1513_result = model_01.predict(tile_1513)
# tile_1514_result = model_01.predict(tile_1514)
# tile_1515_result = model_01.predict(tile_1515)
# tile_1516_result = model_01.predict(tile_1516)
# tile_1517_result = model_01.predict(tile_1517)
# tile_1520_result = model_01.predict(tile_1520)
# tile_1521_result = model_01.predict(tile_1521)
# tile_1529_result = model_01.predict(tile_1529)
# tile_1530_result = model_01.predict(tile_1530)
# tile_1531_result = model_01.predict(tile_1531)
# tile_1532_result = model_01.predict(tile_1532)
# tile_1533_result = model_01.predict(tile_1533)
# tile_1534_result = model_01.predict(tile_1534)
# tile_1535_result = model_01.predict(tile_1535)
# tile_1536_result = model_01.predict(tile_1536)
# tile_1537_result = model_01.predict(tile_1537)
# tile_1538_result = model_01.predict(tile_1538)
# tile_1539_result = model_01.predict(tile_1539)
# tile_1545_result = model_01.predict(tile_1545)
# tile_1546_result = model_01.predict(tile_1546)
# tile_1547_result = model_01.predict(tile_1547)
# tile_1548_result = model_01.predict(tile_1548)
# tile_1549_result = model_01.predict(tile_1549)
# tile_1550_result = model_01.predict(tile_1550)
# tile_1551_result = model_01.predict(tile_1551)
# tile_1555_result = model_01.predict(tile_1555)
# tile_1561_result = model_01.predict(tile_1561)
# tile_1562_result = model_01.predict(tile_1562)
# tile_1563_result = model_01.predict(tile_1563)
# tile_1564_result = model_01.predict(tile_1564)
# tile_1565_result = model_01.predict(tile_1565)
# tile_1571_result = model_01.predict(tile_1571)
# tile_1577_result = model_01.predict(tile_1577)
# tile_1578_result = model_01.predict(tile_1578)
# tile_1579_result = model_01.predict(tile_1579)
# tile_1580_result = model_01.predict(tile_1580)
# tile_1593_result = model_01.predict(tile_1593)
# tile_1594_result = model_01.predict(tile_1594)
# tile_1595_result = model_01.predict(tile_1595)
# tile_1596_result = model_01.predict(tile_1596)
# tile_1610_result = model_01.predict(tile_1610)
# tile_1611_result = model_01.predict(tile_1611)
# tile_1612_result = model_01.predict(tile_1612)
# tile_1628_result = model_01.predict(tile_1628)
# tile_1645_result = model_01.predict(tile_1645)

1/1 [==============================] - 4s 4s/step


In [260]:
# tile_1465_result = model_02.predict(tile_1465)
# tile_1466_result = model_02.predict(tile_1466)
# tile_1472_result = model_02.predict(tile_1472)
# tile_1481_result = model_02.predict(tile_1481)
# tile_1482_result = model_02.predict(tile_1482)
# tile_1483_result = model_02.predict(tile_1483)
# tile_1488_result = model_02.predict(tile_1488)
# tile_1489_result = model_02.predict(tile_1489)
# tile_1490_result = model_02.predict(tile_1490)
# tile_1497_result = model_02.predict(tile_1497)
# tile_1498_result = model_02.predict(tile_1498)
# tile_1499_result = model_02.predict(tile_1499)
# tile_1504_result = model_02.predict(tile_1504)
# tile_1505_result = model_02.predict(tile_1505)
# tile_1507_result = model_02.predict(tile_1507)
# tile_1513_result = model_02.predict(tile_1513)
# tile_1514_result = model_02.predict(tile_1514)
# tile_1515_result = model_02.predict(tile_1515)
# tile_1516_result = model_02.predict(tile_1516)
# tile_1517_result = model_02.predict(tile_1517)
# tile_1520_result = model_02.predict(tile_1520)
# tile_1521_result = model_02.predict(tile_1521)
# tile_1529_result = model_02.predict(tile_1529)
# tile_1530_result = model_02.predict(tile_1530)
# tile_1531_result = model_02.predict(tile_1531)
# tile_1532_result = model_02.predict(tile_1532)
# tile_1533_result = model_02.predict(tile_1533)
# tile_1534_result = model_02.predict(tile_1534)
# tile_1535_result = model_02.predict(tile_1535)
# tile_1536_result = model_02.predict(tile_1536)
# tile_1537_result = model_02.predict(tile_1537)
# tile_1538_result = model_02.predict(tile_1538)
# tile_1539_result = model_02.predict(tile_1539)
# tile_1545_result = model_02.predict(tile_1545)
# tile_1546_result = model_02.predict(tile_1546)
# tile_1547_result = model_02.predict(tile_1547)
# tile_1548_result = model_02.predict(tile_1548)
# tile_1549_result = model_02.predict(tile_1549)
# tile_1550_result = model_02.predict(tile_1550)
# tile_1551_result = model_02.predict(tile_1551)
# tile_1555_result = model_02.predict(tile_1555)
# tile_1561_result = model_02.predict(tile_1561)
# tile_1562_result = model_02.predict(tile_1562)
# tile_1563_result = model_02.predict(tile_1563)
# tile_1564_result = model_02.predict(tile_1564)
# tile_1565_result = model_02.predict(tile_1565)
# tile_1571_result = model_02.predict(tile_1571)
# tile_1577_result = model_02.predict(tile_1577)
# tile_1578_result = model_02.predict(tile_1578)
# tile_1579_result = model_02.predict(tile_1579)
# tile_1580_result = model_02.predict(tile_1580)
# tile_1593_result = model_02.predict(tile_1593)
# tile_1594_result = model_02.predict(tile_1594)
# tile_1595_result = model_02.predict(tile_1595)
# tile_1596_result = model_02.predict(tile_1596)
# tile_1610_result = model_02.predict(tile_1610)
# tile_1611_result = model_02.predict(tile_1611)
# tile_1612_result = model_02.predict(tile_1612)
# tile_1628_result = model_02.predict(tile_1628)
# tile_1645_result = model_02.predict(tile_1645)

1/1 [==============================] - 5s 5s/step


In [269]:
tile_1465_result = model_03.predict(tile_1465)
tile_1466_result = model_03.predict(tile_1466)
tile_1472_result = model_03.predict(tile_1472)
tile_1481_result = model_03.predict(tile_1481)
tile_1482_result = model_03.predict(tile_1482)
tile_1483_result = model_03.predict(tile_1483)
tile_1488_result = model_03.predict(tile_1488)
tile_1489_result = model_03.predict(tile_1489)
tile_1490_result = model_03.predict(tile_1490)
tile_1497_result = model_03.predict(tile_1497)
tile_1498_result = model_03.predict(tile_1498)
tile_1499_result = model_03.predict(tile_1499)
tile_1504_result = model_03.predict(tile_1504)
tile_1505_result = model_03.predict(tile_1505)
tile_1507_result = model_03.predict(tile_1507)
tile_1513_result = model_03.predict(tile_1513)
tile_1514_result = model_03.predict(tile_1514)
tile_1515_result = model_03.predict(tile_1515)
tile_1516_result = model_03.predict(tile_1516)
tile_1517_result = model_03.predict(tile_1517)
tile_1520_result = model_03.predict(tile_1520)
tile_1521_result = model_03.predict(tile_1521)
tile_1529_result = model_03.predict(tile_1529)
tile_1530_result = model_03.predict(tile_1530)
tile_1531_result = model_03.predict(tile_1531)
tile_1532_result = model_03.predict(tile_1532)
tile_1533_result = model_03.predict(tile_1533)
tile_1534_result = model_03.predict(tile_1534)
tile_1535_result = model_03.predict(tile_1535)
tile_1536_result = model_03.predict(tile_1536)
tile_1537_result = model_03.predict(tile_1537)
tile_1538_result = model_03.predict(tile_1538)
tile_1539_result = model_03.predict(tile_1539)
tile_1545_result = model_03.predict(tile_1545)
tile_1546_result = model_03.predict(tile_1546)
tile_1547_result = model_03.predict(tile_1547)
tile_1548_result = model_03.predict(tile_1548)
tile_1549_result = model_03.predict(tile_1549)
tile_1550_result = model_03.predict(tile_1550)
tile_1551_result = model_03.predict(tile_1551)
tile_1555_result = model_03.predict(tile_1555)
tile_1561_result = model_03.predict(tile_1561)
tile_1562_result = model_03.predict(tile_1562)
tile_1563_result = model_03.predict(tile_1563)
tile_1564_result = model_03.predict(tile_1564)
tile_1565_result = model_03.predict(tile_1565)
tile_1571_result = model_03.predict(tile_1571)
tile_1577_result = model_03.predict(tile_1577)
tile_1578_result = model_03.predict(tile_1578)
tile_1579_result = model_03.predict(tile_1579)
tile_1580_result = model_03.predict(tile_1580)
tile_1593_result = model_03.predict(tile_1593)
tile_1594_result = model_03.predict(tile_1594)
tile_1595_result = model_03.predict(tile_1595)
tile_1596_result = model_03.predict(tile_1596)
tile_1610_result = model_03.predict(tile_1610)
tile_1611_result = model_03.predict(tile_1611)
tile_1612_result = model_03.predict(tile_1612)
tile_1628_result = model_03.predict(tile_1628)
tile_1645_result = model_03.predict(tile_1645)

1/1 [==============================] - 4s 4s/step


In [247]:
# tile_742_result = model_01.predict(tile_742)
# tile_743_result = model_01.predict(tile_743)
# tile_759_result = model_01.predict(tile_759)
# tile_760_result = model_01.predict(tile_760)
# tile_761_result = model_01.predict(tile_761)
# tile_762_result = model_01.predict(tile_762)
# tile_776_result = model_01.predict(tile_776)
# tile_777_result = model_01.predict(tile_777)
# tile_778_result = model_01.predict(tile_778)
# tile_779_result = model_01.predict(tile_779)
# tile_780_result = model_01.predict(tile_780)
# tile_781_result = model_01.predict(tile_781)
# tile_792_result = model_01.predict(tile_792)
# tile_793_result = model_01.predict(tile_793)
# tile_794_result = model_01.predict(tile_794)
# tile_795_result = model_01.predict(tile_795)
# tile_796_result = model_01.predict(tile_796)
# tile_797_result = model_01.predict(tile_797)
# tile_798_result = model_01.predict(tile_798)
# tile_805_result = model_01.predict(tile_805)
# tile_808_result = model_01.predict(tile_808)
# tile_809_result = model_01.predict(tile_809)
# tile_810_result = model_01.predict(tile_810)
# tile_811_result = model_01.predict(tile_811)
# tile_812_result = model_01.predict(tile_812)
# tile_813_result = model_01.predict(tile_813)
# tile_815_result = model_01.predict(tile_815)
# tile_821_result = model_01.predict(tile_821)
# tile_823_result = model_01.predict(tile_823)
# tile_825_result = model_01.predict(tile_825)
# tile_826_result = model_01.predict(tile_826)
# tile_827_result = model_01.predict(tile_827)
# tile_828_result = model_01.predict(tile_828)
# tile_829_result = model_01.predict(tile_829)
# tile_830_result = model_01.predict(tile_830)
# tile_831_result = model_01.predict(tile_831)
# tile_841_result = model_01.predict(tile_841)
# tile_842_result = model_01.predict(tile_842)
# tile_843_result = model_01.predict(tile_843)
# tile_845_result = model_01.predict(tile_845)
# tile_847_result = model_01.predict(tile_847)
# tile_848_result = model_01.predict(tile_848)
# tile_849_result = model_01.predict(tile_849)
# tile_860_result = model_01.predict(tile_860)
# tile_861_result = model_01.predict(tile_861)
# tile_862_result = model_01.predict(tile_862)
# tile_865_result = model_01.predict(tile_865)
# tile_866_result = model_01.predict(tile_866)

1/1 [==============================] - 3s 3s/step


In [239]:
# tile_742_result = model_02.predict(tile_742)
# tile_743_result = model_02.predict(tile_743)
# tile_759_result = model_02.predict(tile_759)
# tile_760_result = model_02.predict(tile_760)
# tile_761_result = model_02.predict(tile_761)
# tile_762_result = model_02.predict(tile_762)
# tile_776_result = model_02.predict(tile_776)
# tile_777_result = model_02.predict(tile_777)
# tile_778_result = model_02.predict(tile_778)
# tile_779_result = model_02.predict(tile_779)
# tile_780_result = model_02.predict(tile_780)
# tile_781_result = model_02.predict(tile_781)
# tile_792_result = model_02.predict(tile_792)
# tile_793_result = model_02.predict(tile_793)
# tile_794_result = model_02.predict(tile_794)
# tile_795_result = model_02.predict(tile_795)
# tile_796_result = model_02.predict(tile_796)
# tile_797_result = model_02.predict(tile_797)
# tile_798_result = model_02.predict(tile_798)
# tile_805_result = model_02.predict(tile_805)
# tile_808_result = model_02.predict(tile_808)
# tile_809_result = model_02.predict(tile_809)
# tile_810_result = model_02.predict(tile_810)
# tile_811_result = model_02.predict(tile_811)
# tile_812_result = model_02.predict(tile_812)
# tile_813_result = model_02.predict(tile_813)
# tile_815_result = model_02.predict(tile_815)
# tile_821_result = model_02.predict(tile_821)
# tile_823_result = model_02.predict(tile_823)
# tile_825_result = model_02.predict(tile_825)
# tile_826_result = model_02.predict(tile_826)
# tile_827_result = model_02.predict(tile_827)
# tile_828_result = model_02.predict(tile_828)
# tile_829_result = model_02.predict(tile_829)
# tile_830_result = model_02.predict(tile_830)
# tile_831_result = model_02.predict(tile_831)
# tile_841_result = model_02.predict(tile_841)
# tile_842_result = model_02.predict(tile_842)
# tile_843_result = model_02.predict(tile_843)
# tile_845_result = model_02.predict(tile_845)
# tile_847_result = model_02.predict(tile_847)
# tile_848_result = model_02.predict(tile_848)
# tile_849_result = model_02.predict(tile_849)
# tile_860_result = model_02.predict(tile_860)
# tile_861_result = model_02.predict(tile_861)
# tile_862_result = model_02.predict(tile_862)
# tile_865_result = model_02.predict(tile_865)
# tile_866_result = model_02.predict(tile_866)

1/1 [==============================] - 5s 5s/step


In [270]:

!mkdir kawainui_output_03
!mkdir kawainui_weight_03

# tiles = [742, 743, 759, 760, 761, 762, 776, 777, 778, 779, 780, 781, 792, 793, 794, 795, 796, 797, 798, 805, 808, 809, 810, 811, 812, 813, 815, 821, 823, 825, 826, 827, 828, 829, 830, 831, 841, 842, 843, 845, 847, 848, 849, 860, 861, 862, 865, 866]
tiles = [1465, 1466, 1472, 1481, 1482, 1483, 1488, 1489, 1490, 1497, 1498, 1499, 1504, 1505, 1507, 1513, 1514, 1515, 1516, 1517, 1520, 1521, 1529, 1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1538, 1539, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1555, 1561, 1562, 1563, 1564, 1565, 1571, 1577, 1578, 1579, 1580, 1593, 1594, 1595, 1596, 1610, 1611, 1612, 1628, 1645]

for tile in tiles:
    input_tile = f"/content/kawainui_3year/{tile}_3year.tif"
    output_tile = f"/content/kawainui_output_03/{tile}_3year_out.tif"
    weight_tile = f"/content/kawainui_weight_03/{tile}_3year_weight.tif"

    tile_result = eval(f'tile_{tile}_result')

    with rasterio.open(input_tile) as src:
        out_meta = src.meta.copy()
        out_meta.update({
            'dtype': 'float32',
            'count' : 7
        })
        data_out = tile_result[0]
        data_out = np.moveaxis(data_out, -1, 0)
        weight_matrix = create_weight_matrix(384, 384)
        weighted_data_out = weight_matrix * data_out

    with rasterio.open(output_tile, 'w', **out_meta) as dst:
        dst.write(data_out)
    with rasterio.open(weight_tile, 'w', **out_meta) as dst:
        dst.write(weighted_data_out)

In [271]:
import os
import rasterio
from rasterio.merge import merge

def gather_raster_files(*directories):
    """Gather raster files from provided directories."""
    raster_paths = []
    for directory in directories:
        # List all TIFF files in each directory
        for filename in os.listdir(directory):
            if filename.endswith('.tif'):
                raster_paths.append(os.path.join(directory, filename))
    return raster_paths

def create_mosaic_sum(input_rasters, output_filename='mosaic_sum.tif'):
    """Create a mosaic from the sum of overlapping bands using rasterio."""
    src_files = [rasterio.open(path) for path in input_rasters]

    # Use rasterio's merge function with method='sum' to sum overlapping pixels
    mosaic, out_trans = merge(src_files, method='sum')

    # Define the metadata for the output GeoTIFF
    out_meta = src_files[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "count": src_files[0].count
    })

    # Write the mosaic raster to disk
    with rasterio.open(output_filename, 'w', **out_meta) as dest:
        dest.write(mosaic)

    # Close all raster files
    for src in src_files:
        src.close()

    print(f"Mosaic created successfully: {output_filename}")
    return mosaic, out_trans

# Directories containing the raster files
output_dir = "/content/kawainui_output_03"
weighted_dir = "/content/kawainui_weight_03"

# Gather all raster files from the directories
raster_files = gather_raster_files(output_dir, weighted_dir)

# Create a mosaic with the sum of overlapping bands
mosaic, out_transform = create_mosaic_sum(raster_files, 'kawainui_03_mosaic.tif')

Mosaic created successfully: kawainui_03_mosaic.tif


In [225]:
# tile_742 = read_thirteen_band_image("/content/kahuku_3year/742_3year.tif")
# tile_743 = read_thirteen_band_image("/content/kahuku_3year/743_3year.tif")
# tile_759 = read_thirteen_band_image("/content/kahuku_3year/759_3year.tif")
# tile_760 = read_thirteen_band_image("/content/kahuku_3year/760_3year.tif")
# tile_761 = read_thirteen_band_image("/content/kahuku_3year/761_3year.tif")
# tile_762 = read_thirteen_band_image("/content/kahuku_3year/762_3year.tif")
# tile_776 = read_thirteen_band_image("/content/kahuku_3year/776_3year.tif")
# tile_777 = read_thirteen_band_image("/content/kahuku_3year/777_3year.tif")
# tile_778 = read_thirteen_band_image("/content/kahuku_3year/778_3year.tif")
# tile_779 = read_thirteen_band_image("/content/kahuku_3year/779_3year.tif")
# tile_780 = read_thirteen_band_image("/content/kahuku_3year/780_3year.tif")
# tile_781 = read_thirteen_band_image("/content/kahuku_3year/781_3year.tif")
# tile_792 = read_thirteen_band_image("/content/kahuku_3year/792_3year.tif")
# tile_793 = read_thirteen_band_image("/content/kahuku_3year/793_3year.tif")
# tile_794 = read_thirteen_band_image("/content/kahuku_3year/794_3year.tif")
# tile_795 = read_thirteen_band_image("/content/kahuku_3year/795_3year.tif")
# tile_796 = read_thirteen_band_image("/content/kahuku_3year/796_3year.tif")
# tile_797 = read_thirteen_band_image("/content/kahuku_3year/797_3year.tif")
# tile_798 = read_thirteen_band_image("/content/kahuku_3year/798_3year.tif")
# tile_805 = read_thirteen_band_image("/content/kahuku_3year/805_3year.tif")
# tile_808 = read_thirteen_band_image("/content/kahuku_3year/808_3year.tif")
# tile_809 = read_thirteen_band_image("/content/kahuku_3year/809_3year.tif")
# tile_810 = read_thirteen_band_image("/content/kahuku_3year/810_3year.tif")
# tile_811 = read_thirteen_band_image("/content/kahuku_3year/811_3year.tif")
# tile_812 = read_thirteen_band_image("/content/kahuku_3year/812_3year.tif")
# tile_813 = read_thirteen_band_image("/content/kahuku_3year/813_3year.tif")
# tile_815 = read_thirteen_band_image("/content/kahuku_3year/815_3year.tif")
# tile_821 = read_thirteen_band_image("/content/kahuku_3year/821_3year.tif")
# tile_823 = read_thirteen_band_image("/content/kahuku_3year/823_3year.tif")
# tile_825 = read_thirteen_band_image("/content/kahuku_3year/825_3year.tif")
# tile_826 = read_thirteen_band_image("/content/kahuku_3year/826_3year.tif")
# tile_827 = read_thirteen_band_image("/content/kahuku_3year/827_3year.tif")
# tile_828 = read_thirteen_band_image("/content/kahuku_3year/828_3year.tif")
# tile_829 = read_thirteen_band_image("/content/kahuku_3year/829_3year.tif")
# tile_830 = read_thirteen_band_image("/content/kahuku_3year/830_3year.tif")
# tile_831 = read_thirteen_band_image("/content/kahuku_3year/831_3year.tif")
# tile_841 = read_thirteen_band_image("/content/kahuku_3year/841_3year.tif")
# tile_842 = read_thirteen_band_image("/content/kahuku_3year/842_3year.tif")
# tile_843 = read_thirteen_band_image("/content/kahuku_3year/843_3year.tif")
# tile_845 = read_thirteen_band_image("/content/kahuku_3year/845_3year.tif")
# tile_847 = read_thirteen_band_image("/content/kahuku_3year/847_3year.tif")
# tile_848 = read_thirteen_band_image("/content/kahuku_3year/848_3year.tif")
# tile_849 = read_thirteen_band_image("/content/kahuku_3year/849_3year.tif")
# tile_860 = read_thirteen_band_image("/content/kahuku_3year/860_3year.tif")
# tile_861 = read_thirteen_band_image("/content/kahuku_3year/861_3year.tif")
# tile_862 = read_thirteen_band_image("/content/kahuku_3year/862_3year.tif")
# tile_865 = read_thirteen_band_image("/content/kahuku_3year/865_3year.tif")
# tile_866 = read_thirteen_band_image("/content/kahuku_3year/866_3year.tif")




In [ ]:
tile_742 tile_743 tile_759 tile_760 tile_761 tile_762 tile_776 tile_777 tile_778 tile_779 tile_780 tile_781
tile_792 tile_793 tile_794 tile_795 tile_796 tile_797 tile_798 tile_805 tile_808 tile_809 tile_810 tile_811
tile_812 tile_813 tile_815 tile_821 tile_823 tile_825 tile_826 tile_827 tile_828 tile_829 tile_830 tile_831
tile_841 tile_842 tile_843 tile_845 tile_847 tile_848 tile_849 tile_860 tile_861 tile_862 tile_865 tile_866

In [231]:
# tile_742_result = model_03.predict(tile_742)
# tile_743_result = model_03.predict(tile_743)
# tile_759_result = model_03.predict(tile_759)
# tile_760_result = model_03.predict(tile_760)
# tile_761_result = model_03.predict(tile_761)
# tile_762_result = model_03.predict(tile_762)
# tile_776_result = model_03.predict(tile_776)
# tile_777_result = model_03.predict(tile_777)
# tile_778_result = model_03.predict(tile_778)
# tile_779_result = model_03.predict(tile_779)
# tile_780_result = model_03.predict(tile_780)
# tile_781_result = model_03.predict(tile_781)
# tile_792_result = model_03.predict(tile_792)
# tile_793_result = model_03.predict(tile_793)
# tile_794_result = model_03.predict(tile_794)
# tile_795_result = model_03.predict(tile_795)
# tile_796_result = model_03.predict(tile_796)
# tile_797_result = model_03.predict(tile_797)
# tile_798_result = model_03.predict(tile_798)
# tile_805_result = model_03.predict(tile_805)
# tile_808_result = model_03.predict(tile_808)
# tile_809_result = model_03.predict(tile_809)
# tile_810_result = model_03.predict(tile_810)
# tile_811_result = model_03.predict(tile_811)
# tile_812_result = model_03.predict(tile_812)
# tile_813_result = model_03.predict(tile_813)
# tile_815_result = model_03.predict(tile_815)
# tile_821_result = model_03.predict(tile_821)
# tile_823_result = model_03.predict(tile_823)
# tile_825_result = model_03.predict(tile_825)
# tile_826_result = model_03.predict(tile_826)
# tile_827_result = model_03.predict(tile_827)
# tile_828_result = model_03.predict(tile_828)
# tile_829_result = model_03.predict(tile_829)
# tile_830_result = model_03.predict(tile_830)
# tile_831_result = model_03.predict(tile_831)
# tile_841_result = model_03.predict(tile_841)
# tile_842_result = model_03.predict(tile_842)
# tile_843_result = model_03.predict(tile_843)
# tile_845_result = model_03.predict(tile_845)
# tile_847_result = model_03.predict(tile_847)
# tile_848_result = model_03.predict(tile_848)
# tile_849_result = model_03.predict(tile_849)
# tile_860_result = model_03.predict(tile_860)
# tile_861_result = model_03.predict(tile_861)
# tile_862_result = model_03.predict(tile_862)
# tile_865_result = model_03.predict(tile_865)
# tile_866_result = model_03.predict(tile_866)


1/1 [==============================] - 4s 4s/step


In [168]:
# # prompt: using this as an example, tile_111 = read_nine_band_image("/content/haleiwa_02/111_2year.tif")
# # change read_nine_band_image to read_thirteen_band_image for all the files. all the files

# tile_111 = read_thirteen_band_image("/content/haleiwa_03/111_3year.tif")
# tile_112 = read_thirteen_band_image("/content/haleiwa_03/112_3year.tif")
# tile_126 = read_thirteen_band_image("/content/haleiwa_03/126_3year.tif")
# tile_127 = read_thirteen_band_image("/content/haleiwa_03/127_3year.tif")
# tile_140 = read_thirteen_band_image("/content/haleiwa_03/140_3year.tif")
# tile_141 = read_thirteen_band_image("/content/haleiwa_03/141_3year.tif")
# tile_142 = read_thirteen_band_image("/content/haleiwa_03/142_3year.tif")
# tile_143 = read_thirteen_band_image("/content/haleiwa_03/143_3year.tif")
# tile_154 = read_thirteen_band_image("/content/haleiwa_03/154_3year.tif")
# tile_155 = read_thirteen_band_image("/content/haleiwa_03/155_3year.tif")
# tile_156 = read_thirteen_band_image("/content/haleiwa_03/156_3year.tif")
# tile_157 = read_thirteen_band_image("/content/haleiwa_03/157_3year.tif")
# tile_158 = read_thirteen_band_image("/content/haleiwa_03/158_3year.tif")
# tile_159 = read_thirteen_band_image("/content/haleiwa_03/159_3year.tif")
# tile_170 = read_thirteen_band_image("/content/haleiwa_03/170_3year.tif")
# tile_171 = read_thirteen_band_image("/content/haleiwa_03/171_3year.tif")
# tile_172 = read_thirteen_band_image("/content/haleiwa_03/172_3year.tif")
# tile_173 = read_thirteen_band_image("/content/haleiwa_03/173_3year.tif")
# tile_185 = read_thirteen_band_image("/content/haleiwa_03/185_3year.tif")
# tile_186 = read_thirteen_band_image("/content/haleiwa_03/186_3year.tif")
# tile_188 = read_thirteen_band_image("/content/haleiwa_03/188_3year.tif")
# tile_189 = read_thirteen_band_image("/content/haleiwa_03/189_3year.tif")
# tile_203 = read_thirteen_band_image("/content/haleiwa_03/203_3year.tif")
# tile_204 = read_thirteen_band_image("/content/haleiwa_03/204_3year.tif")
# tile_219 = read_thirteen_band_image("/content/haleiwa_03/219_3year.tif")
# tile_220 = read_thirteen_band_image("/content/haleiwa_03/220_3year.tif")
# tile_221 = read_thirteen_band_image("/content/haleiwa_03/221_3year.tif")
# tile_222 = read_thirteen_band_image("/content/haleiwa_03/222_3year.tif")
# tile_234 = read_thirteen_band_image("/content/haleiwa_03/234_3year.tif")
# tile_235 = read_thirteen_band_image("/content/haleiwa_03/235_3year.tif")
# tile_250 = read_thirteen_band_image("/content/haleiwa_03/250_3year.tif")
# tile_251 = read_thirteen_band_image("/content/haleiwa_03/251_3year.tif")
# tile_252 = read_thirteen_band_image("/content/haleiwa_03/252_3year.tif")
# tile_265 = read_thirteen_band_image("/content/haleiwa_03/265_3year.tif")
# tile_266 = read_thirteen_band_image("/content/haleiwa_03/266_3year.tif")
# tile_267 = read_thirteen_band_image("/content/haleiwa_03/267_3year.tif")
# tile_280 = read_thirteen_band_image("/content/haleiwa_03/280_3year.tif")
# tile_281 = read_thirteen_band_image("/content/haleiwa_03/281_3year.tif")
# tile_282 = read_thirteen_band_image("/content/haleiwa_03/282_3year.tif")
# tile_283 = read_thirteen_band_image("/content/haleiwa_03/283_3year.tif")
# tile_298 = read_thirteen_band_image("/content/haleiwa_03/298_3year.tif")
# tile_330 = read_thirteen_band_image("/content/haleiwa_03/330_3year.tif")

In [109]:
# tile_111 = read_nine_band_image("/content/haleiwa_02/111_2year.tif")
# tile_112 = read_nine_band_image("/content/haleiwa_02/112_2year.tif")
# tile_126 = read_nine_band_image("/content/haleiwa_02/126_2year.tif")
# tile_127 = read_nine_band_image("/content/haleiwa_02/127_2year.tif")
# tile_140 = read_nine_band_image("/content/haleiwa_02/140_2year.tif")
# tile_141 = read_nine_band_image("/content/haleiwa_02/141_2year.tif")
# tile_142 = read_nine_band_image("/content/haleiwa_02/142_2year.tif")
# tile_143 = read_nine_band_image("/content/haleiwa_02/143_2year.tif")
# tile_154 = read_nine_band_image("/content/haleiwa_02/154_2year.tif")
# tile_155 = read_nine_band_image("/content/haleiwa_02/155_2year.tif")
# tile_156 = read_nine_band_image("/content/haleiwa_02/156_2year.tif")
# tile_157 = read_nine_band_image("/content/haleiwa_02/157_2year.tif")
# tile_158 = read_nine_band_image("/content/haleiwa_02/158_2year.tif")
# tile_159 = read_nine_band_image("/content/haleiwa_02/159_2year.tif")
# tile_170 = read_nine_band_image("/content/haleiwa_02/170_2year.tif")
# tile_171 = read_nine_band_image("/content/haleiwa_02/171_2year.tif")
# tile_172 = read_nine_band_image("/content/haleiwa_02/172_2year.tif")
# tile_173 = read_nine_band_image("/content/haleiwa_02/173_2year.tif")
# tile_185 = read_nine_band_image("/content/haleiwa_02/185_2year.tif")
# tile_186 = read_nine_band_image("/content/haleiwa_02/186_2year.tif")
# tile_188 = read_nine_band_image("/content/haleiwa_02/188_2year.tif")
# tile_189 = read_nine_band_image("/content/haleiwa_02/189_2year.tif")
# tile_203 = read_nine_band_image("/content/haleiwa_02/203_2year.tif")
# tile_204 = read_nine_band_image("/content/haleiwa_02/204_2year.tif")
# tile_219 = read_nine_band_image("/content/haleiwa_02/219_2year.tif")
# tile_220 = read_nine_band_image("/content/haleiwa_02/220_2year.tif")
# tile_221 = read_nine_band_image("/content/haleiwa_02/221_2year.tif")
# tile_222 = read_nine_band_image("/content/haleiwa_02/222_2year.tif")
# tile_234 = read_nine_band_image("/content/haleiwa_02/234_2year.tif")
# tile_235 = read_nine_band_image("/content/haleiwa_02/235_2year.tif")
# tile_250 = read_nine_band_image("/content/haleiwa_02/250_2year.tif")
# tile_251 = read_nine_band_image("/content/haleiwa_02/251_2year.tif")
# tile_252 = read_nine_band_image("/content/haleiwa_02/252_2year.tif")
# tile_265 = read_nine_band_image("/content/haleiwa_02/265_2year.tif")
# tile_266 = read_nine_band_image("/content/haleiwa_02/266_2year.tif")
# tile_267 = read_nine_band_image("/content/haleiwa_02/267_2year.tif")
# tile_280 = read_nine_band_image("/content/haleiwa_02/280_2year.tif")
# tile_281 = read_nine_band_image("/content/haleiwa_02/281_2year.tif")
# tile_282 = read_nine_band_image("/content/haleiwa_02/282_2year.tif")
# tile_283 = read_nine_band_image("/content/haleiwa_02/283_2year.tif")
# tile_298 = read_nine_band_image("/content/haleiwa_02/298_2year.tif")
# tile_330 = read_nine_band_image("/content/haleiwa_02/330_2year.tif")

In [60]:
# tile_111 = read_multi_image_withfive("/content/haleiwa_01/111_1year.tif")
# tile_112 = read_multi_image_withfive("/content/haleiwa_01/112_1year.tif")
# tile_126 = read_multi_image_withfive("/content/haleiwa_01/126_1year.tif")
# tile_127 = read_multi_image_withfive("/content/haleiwa_01/127_1year.tif")
# tile_140 = read_multi_image_withfive("/content/haleiwa_01/140_1year.tif")
# tile_141 = read_multi_image_withfive("/content/haleiwa_01/141_1year.tif")
# tile_142 = read_multi_image_withfive("/content/haleiwa_01/142_1year.tif")
# tile_143 = read_multi_image_withfive("/content/haleiwa_01/143_1year.tif")
# tile_154 = read_multi_image_withfive("/content/haleiwa_01/154_1year.tif")
# tile_155 = read_multi_image_withfive("/content/haleiwa_01/155_1year.tif")
# tile_156 = read_multi_image_withfive("/content/haleiwa_01/156_1year.tif")
# tile_157 = read_multi_image_withfive("/content/haleiwa_01/157_1year.tif")
# tile_158 = read_multi_image_withfive("/content/haleiwa_01/158_1year.tif")
# tile_159 = read_multi_image_withfive("/content/haleiwa_01/159_1year.tif")
# tile_170 = read_multi_image_withfive("/content/haleiwa_01/170_1year.tif")
# tile_171 = read_multi_image_withfive("/content/haleiwa_01/171_1year.tif")
# tile_172 = read_multi_image_withfive("/content/haleiwa_01/172_1year.tif")
# tile_173 = read_multi_image_withfive("/content/haleiwa_01/173_1year.tif")
# tile_185 = read_multi_image_withfive("/content/haleiwa_01/185_1year.tif")
# tile_186 = read_multi_image_withfive("/content/haleiwa_01/186_1year.tif")
# tile_188 = read_multi_image_withfive("/content/haleiwa_01/188_1year.tif")
# tile_189 = read_multi_image_withfive("/content/haleiwa_01/189_1year.tif")
# tile_203 = read_multi_image_withfive("/content/haleiwa_01/203_1year.tif")
# tile_204 = read_multi_image_withfive("/content/haleiwa_01/204_1year.tif")
# tile_219 = read_multi_image_withfive("/content/haleiwa_01/219_1year.tif")
# tile_220 = read_multi_image_withfive("/content/haleiwa_01/220_1year.tif")
# tile_221 = read_multi_image_withfive("/content/haleiwa_01/221_1year.tif")
# tile_222 = read_multi_image_withfive("/content/haleiwa_01/222_1year.tif")
# tile_234 = read_multi_image_withfive("/content/haleiwa_01/234_1year.tif")
# tile_235 = read_multi_image_withfive("/content/haleiwa_01/235_1year.tif")
# tile_250 = read_multi_image_withfive("/content/haleiwa_01/250_1year.tif")
# tile_251 = read_multi_image_withfive("/content/haleiwa_01/251_1year.tif")
# tile_252 = read_multi_image_withfive("/content/haleiwa_01/252_1year.tif")
# tile_265 = read_multi_image_withfive("/content/haleiwa_01/265_1year.tif")
# tile_266 = read_multi_image_withfive("/content/haleiwa_01/266_1year.tif")
# tile_267 = read_multi_image_withfive("/content/haleiwa_01/267_1year.tif")
# tile_280 = read_multi_image_withfive("/content/haleiwa_01/280_1year.tif")
# tile_281 = read_multi_image_withfive("/content/haleiwa_01/281_1year.tif")
# tile_282 = read_multi_image_withfive("/content/haleiwa_01/282_1year.tif")
# tile_283 = read_multi_image_withfive("/content/haleiwa_01/283_1year.tif")
# tile_298 = read_multi_image_withfive("/content/haleiwa_01/298_1year.tif")
# tile_330 = read_multi_image_withfive("/content/haleiwa_01/330_1year.tif")



In [169]:
# tile_111 = np.expand_dims(tile_111, axis=0)
# tile_112 = np.expand_dims(tile_112, axis=0)
# tile_126 = np.expand_dims(tile_126, axis=0)
# tile_127 = np.expand_dims(tile_127, axis=0)
# tile_140 = np.expand_dims(tile_140, axis=0)
# tile_141 = np.expand_dims(tile_141, axis=0)
# tile_142 = np.expand_dims(tile_142, axis=0)
# tile_143 = np.expand_dims(tile_143, axis=0)
# tile_154 = np.expand_dims(tile_154, axis=0)
# tile_155 = np.expand_dims(tile_155, axis=0)
# tile_156 = np.expand_dims(tile_156, axis=0)
# tile_157 = np.expand_dims(tile_157, axis=0)
# tile_158 = np.expand_dims(tile_158, axis=0)
# tile_159 = np.expand_dims(tile_159, axis=0)
# tile_170 = np.expand_dims(tile_170, axis=0)
# tile_171 = np.expand_dims(tile_171, axis=0)
# tile_172 = np.expand_dims(tile_172, axis=0)
# tile_173 = np.expand_dims(tile_173, axis=0)
# tile_185 = np.expand_dims(tile_185, axis=0)
# tile_186 = np.expand_dims(tile_186, axis=0)
# tile_188 = np.expand_dims(tile_188, axis=0)
# tile_189 = np.expand_dims(tile_189, axis=0)
# tile_203 = np.expand_dims(tile_203, axis=0)
# tile_204 = np.expand_dims(tile_204, axis=0)
# tile_219 = np.expand_dims(tile_219, axis=0)
# tile_220 = np.expand_dims(tile_220, axis=0)
# tile_221 = np.expand_dims(tile_221, axis=0)
# tile_222 = np.expand_dims(tile_222, axis=0)
# tile_234 = np.expand_dims(tile_234, axis=0)
# tile_235 = np.expand_dims(tile_235, axis=0)
# tile_250 = np.expand_dims(tile_250, axis=0)
# tile_251 = np.expand_dims(tile_251, axis=0)
# tile_252 = np.expand_dims(tile_252, axis=0)
# tile_265 = np.expand_dims(tile_265, axis=0)
# tile_266 = np.expand_dims(tile_266, axis=0)
# tile_267 = np.expand_dims(tile_267, axis=0)
# tile_280 = np.expand_dims(tile_280, axis=0)
# tile_281 = np.expand_dims(tile_281, axis=0)
# tile_282 = np.expand_dims(tile_282, axis=0)
# tile_283 = np.expand_dims(tile_283, axis=0)
# tile_298 = np.expand_dims(tile_298, axis=0)
# tile_330 = np.expand_dims(tile_330, axis=0)

1/1 [==============================] - 5s 5s/step


1/1 [==============================] - 3s 3s/step


In [171]:
# tile_111_result = model_03.predict(tile_111)
# tile_112_result = model_03.predict(tile_112)
# tile_126_result = model_03.predict(tile_126)
# tile_127_result = model_03.predict(tile_127)
# tile_140_result = model_03.predict(tile_140)
# tile_141_result = model_03.predict(tile_141)
# tile_142_result = model_03.predict(tile_142)
# tile_143_result = model_03.predict(tile_143)
# tile_154_result = model_03.predict(tile_154)
# tile_155_result = model_03.predict(tile_155)
# tile_156_result = model_03.predict(tile_156)
# tile_157_result = model_03.predict(tile_157)
# tile_158_result = model_03.predict(tile_158)
# tile_159_result = model_03.predict(tile_159)
# tile_170_result = model_03.predict(tile_170)
# tile_171_result = model_03.predict(tile_171)
# tile_172_result = model_03.predict(tile_172)
# tile_173_result = model_03.predict(tile_173)
# tile_185_result = model_03.predict(tile_185)
# tile_186_result = model_03.predict(tile_186)
# tile_188_result = model_03.predict(tile_188)
# tile_189_result = model_03.predict(tile_189)
# tile_203_result = model_03.predict(tile_203)
# tile_204_result = model_03.predict(tile_204)
# tile_219_result = model_03.predict(tile_219)
# tile_220_result = model_03.predict(tile_220)
# tile_221_result = model_03.predict(tile_221)
# tile_222_result = model_03.predict(tile_222)
# tile_234_result = model_03.predict(tile_234)
# tile_235_result = model_03.predict(tile_235)
# tile_250_result = model_03.predict(tile_250)
# tile_251_result = model_03.predict(tile_251)
# tile_252_result = model_03.predict(tile_252)
# tile_265_result = model_03.predict(tile_265)
# tile_266_result = model_03.predict(tile_266)
# tile_267_result = model_03.predict(tile_267)
# tile_280_result = model_03.predict(tile_280)
# tile_281_result = model_03.predict(tile_281)
# tile_282_result = model_03.predict(tile_282)
# tile_283_result = model_03.predict(tile_283)
# tile_298_result = model_03.predict(tile_298)
# tile_330_result = model_03.predict(tile_330)


1/1 [==============================] - 4s 4s/step


In [ ]:

# weight_tile_111 = "/content/output_weight_01/111_1year_weight.tif"
# weight_tile_112 = "/content/output_weight_01/112_1year_weight.tif"
# weight_tile_126 = "/content/output_weight_01/126_1year_weight.tif"
# weight_tile_127 = "/content/output_weight_01/127_1year_weight.tif"
# weight_tile_140 = "/content/output_weight_01/140_1year_weight.tif"
# weight_tile_141 = "/content/output_weight_01/141_1year_weight.tif"
# weight_tile_142 = "/content/output_weight_01/142_1year_weight.tif"
# weight_tile_143 = "/content/output_weight_01/143_1year_weight.tif"
# weight_tile_154 = "/content/output_weight_01/154_1year_weight.tif"
# weight_tile_155 = "/content/output_weight_01/155_1year_weight.tif"
# weight_tile_156 = "/content/output_weight_01/156_1year_weight.tif"
# weight_tile_157 = "/content/output_weight_01/157_1year_weight.tif"
# weight_tile_158 = "/content/output_weight_01/158_1year_weight.tif"
# weight_tile_159 = "/content/output_weight_01/159_1year_weight.tif"
# weight_tile_170 = "/content/output_weight_01/170_1year_weight.tif"
# weight_tile_171 = "/content/output_weight_01/171_1year_weight.tif"
# weight_tile_172 = "/content/output_weight_01/172_1year_weight.tif"
# weight_tile_173 = "/content/output_weight_01/173_1year_weight.tif"
# weight_tile_185 = "/content/output_weight_01/185_1year_weight.tif"
# weight_tile_186 = "/content/output_weight_01/186_1year_weight.tif"
# weight_tile_188 = "/content/output_weight_01/188_1year_weight.tif"
# weight_tile_189 = "/content/output_weight_01/189_1year_weight.tif"
# weight_tile_203 = "/content/output_weight_01/203_1year_weight.tif"
# weight_tile_204 = "/content/output_weight_01/204_1year_weight.tif"
# weight_tile_219 = "/content/output_weight_01/219_1year_weight.tif"
# weight_tile_220 = "/content/output_weight_01/220_1year_weight.tif"
# weight_tile_221 = "/content/output_weight_01/221_1year_weight.tif"
# weight_tile_222 = "/content/output_weight_01/222_1year_weight.tif"
# weight_tile_234 = "/content/output_weight_01/234_1year_weight.tif"
# weight_tile_235 = "/content/output_weight_01/235_1year_weight.tif"
# weight_tile_250 = "/content/output_weight_01/250_1year_weight.tif"
# weight_tile_251 = "/content/output_weight_01/251_1year_weight.tif"
# weight_tile_252 = "/content/output_weight_01/252_1year_weight.tif"
# weight_tile_265 = "/content/output_weight_01/265_1year_weight.tif"
# weight_tile_266 = "/content/output_weight_01/266_1year_weight.tif"
# weight_tile_267 = "/content/output_weight_01/267_1year_weight.tif"
# weight_tile_280 = "/content/output_weight_01/280_1year_weight.tif"
# weight_tile_281 = "/content/output_weight_01/281_1year_weight.tif"
# weight_tile_282 = "/content/output_weight_01/282_1year_weight.tif"
# weight_tile_283 = "/content/output_weight_01/283_1year_weight.tif"
# weight_tile_298 = "/content/output_weight_01/298_1year_weight.tif"
# weight_tile_330 = "/content/output_weight_01/330_1year_weight.tif"


In [ ]:
# weight_tile_111 = "/content/output_weight_02/111_2year_weight.tif"
# weight_tile_112 = "/content/output_weight_02/112_2year_weight.tif"
# weight_tile_126 = "/content/output_weight_02/126_2year_weight.tif"
# weight_tile_127 = "/content/output_weight_02/127_2year_weight.tif"
# weight_tile_140 = "/content/output_weight_02/140_2year_weight.tif"
# weight_tile_141 = "/content/output_weight_02/141_2year_weight.tif"
# weight_tile_142 = "/content/output_weight_02/142_2year_weight.tif"
# weight_tile_143 = "/content/output_weight_02/143_2year_weight.tif"
# weight_tile_154 = "/content/output_weight_02/154_2year_weight.tif"
# weight_tile_155 = "/content/output_weight_02/155_2year_weight.tif"
# weight_tile_156 = "/content/output_weight_02/156_2year_weight.tif"
# weight_tile_157 = "/content/output_weight_02/157_2year_weight.tif"
# weight_tile_158 = "/content/output_weight_02/158_2year_weight.tif"
# weight_tile_159 = "/content/output_weight_02/159_2year_weight.tif"
# weight_tile_170 = "/content/output_weight_02/170_2year_weight.tif"
# weight_tile_171 = "/content/output_weight_02/171_2year_weight.tif"
# weight_tile_172 = "/content/output_weight_02/172_2year_weight.tif"
# weight_tile_173 = "/content/output_weight_02/173_2year_weight.tif"
# weight_tile_185 = "/content/output_weight_02/185_2year_weight.tif"
# weight_tile_186 = "/content/output_weight_02/186_2year_weight.tif"
# weight_tile_188 = "/content/output_weight_02/188_2year_weight.tif"
# weight_tile_189 = "/content/output_weight_02/189_2year_weight.tif"
# weight_tile_203 = "/content/output_weight_02/203_2year_weight.tif"
# weight_tile_204 = "/content/output_weight_02/204_2year_weight.tif"
# weight_tile_219 = "/content/output_weight_02/219_2year_weight.tif"
# weight_tile_220 = "/content/output_weight_02/220_2year_weight.tif"
# weight_tile_221 = "/content/output_weight_02/221_2year_weight.tif"
# weight_tile_222 = "/content/output_weight_02/222_2year_weight.tif"
# weight_tile_234 = "/content/output_weight_02/234_2year_weight.tif"
# weight_tile_235 = "/content/output_weight_02/235_2year_weight.tif"
# weight_tile_250 = "/content/output_weight_02/250_2year_weight.tif"
# weight_tile_251 = "/content/output_weight_02/251_2year_weight.tif"
# weight_tile_252 = "/content/output_weight_02/252_2year_weight.tif"
# weight_tile_265 = "/content/output_weight_02/265_2year_weight.tif"
# weight_tile_266 = "/content/output_weight_02/266_2year_weight.tif"
# weight_tile_267 = "/content/output_weight_02/267_2year_weight.tif"
# weight_tile_280 = "/content/output_weight_02/280_2year_weight.tif"
# weight_tile_281 = "/content/output_weight_02/281_2year_weight.tif"
# weight_tile_282 = "/content/output_weight_02/282_2year_weight.tif"
# weight_tile_283 = "/content/output_weight_02/283_2year_weight.tif"
# weight_tile_298 = "/content/output_weight_02/298_2year_weight.tif"
# weight_tile_330 = "/content/output_weight_02/330_2year_weight.tif"

In [172]:
# # prompt: change output_weight_02 to output_weight_03

# weight_tile_111 = "/content/output_weight_03/111_3year_weight.tif"
# weight_tile_112 = "/content/output_weight_03/112_3year_weight.tif"
# weight_tile_126 = "/content/output_weight_03/126_3year_weight.tif"
# weight_tile_127 = "/content/output_weight_03/127_3year_weight.tif"
# weight_tile_140 = "/content/output_weight_03/140_3year_weight.tif"
# weight_tile_141 = "/content/output_weight_03/141_3year_weight.tif"
# weight_tile_142 = "/content/output_weight_03/142_3year_weight.tif"
# weight_tile_143 = "/content/output_weight_03/143_3year_weight.tif"
# weight_tile_154 = "/content/output_weight_03/154_3year_weight.tif"
# weight_tile_155 = "/content/output_weight_03/155_3year_weight.tif"
# weight_tile_156 = "/content/output_weight_03/156_3year_weight.tif"
# weight_tile_157 = "/content/output_weight_03/157_3year_weight.tif"
# weight_tile_158 = "/content/output_weight_03/158_3year_weight.tif"
# weight_tile_159 = "/content/output_weight_03/159_3year_weight.tif"
# weight_tile_170 = "/content/output_weight_03/170_3year_weight.tif"
# weight_tile_171 = "/content/output_weight_03/171_3year_weight.tif"
# weight_tile_172 = "/content/output_weight_03/172_3year_weight.tif"
# weight_tile_173 = "/content/output_weight_03/173_3year_weight.tif"
# weight_tile_185 = "/content/output_weight_03/185_3year_weight.tif"
# weight_tile_186 = "/content/output_weight_03/186_3year_weight.tif"
# weight_tile_188 = "/content/output_weight_03/188_3year_weight.tif"
# weight_tile_189 = "/content/output_weight_03/189_3year_weight.tif"
# weight_tile_203 = "/content/output_weight_03/203_3year_weight.tif"
# weight_tile_204 = "/content/output_weight_03/204_3year_weight.tif"
# weight_tile_219 = "/content/output_weight_03/219_3year_weight.tif"
# weight_tile_220 = "/content/output_weight_03/220_3year_weight.tif"
# weight_tile_221 = "/content/output_weight_03/221_3year_weight.tif"
# weight_tile_222 = "/content/output_weight_03/222_3year_weight.tif"
# weight_tile_234 = "/content/output_weight_03/234_3year_weight.tif"
# weight_tile_235 = "/content/output_weight_03/235_3year_weight.tif"
# weight_tile_250 = "/content/output_weight_03/250_3year_weight.tif"
# weight_tile_251 = "/content/output_weight_03/251_3year_weight.tif"
# weight_tile_252 = "/content/output_weight_03/252_3year_weight.tif"
# weight_tile_265 = "/content/output_weight_03/265_3year_weight.tif"
# weight_tile_266 = "/content/output_weight_03/266_3year_weight.tif"
# weight_tile_267 = "/content/output_weight_03/267_3year_weight.tif"
# weight_tile_280 = "/content/output_weight_03/280_3year_weight.tif"
# weight_tile_281 = "/content/output_weight_03/281_3year_weight.tif"
# weight_tile_282 = "/content/output_weight_03/282_3year_weight.tif"
# weight_tile_283 = "/content/output_weight_03/283_3year_weight.tif"
# weight_tile_298 = "/content/output_weight_03/298_3year_weight.tif"
# weight_tile_330 = "/content/output_weight_03/330_3year_weight.tif"


In [173]:
# input_tile_111 = "/content/haleiwa_03/111_3year.tif"
# input_tile_112 = "/content/haleiwa_03/112_3year.tif"
# input_tile_126 = "/content/haleiwa_03/126_3year.tif"
# input_tile_127 = "/content/haleiwa_03/127_3year.tif"
# input_tile_140 = "/content/haleiwa_03/140_3year.tif"
# input_tile_141 = "/content/haleiwa_03/141_3year.tif"
# input_tile_142 = "/content/haleiwa_03/142_3year.tif"
# input_tile_143 = "/content/haleiwa_03/143_3year.tif"
# input_tile_154 = "/content/haleiwa_03/154_3year.tif"
# input_tile_155 = "/content/haleiwa_03/155_3year.tif"
# input_tile_156 = "/content/haleiwa_03/156_3year.tif"
# input_tile_157 = "/content/haleiwa_03/157_3year.tif"
# input_tile_158 = "/content/haleiwa_03/158_3year.tif"
# input_tile_159 = "/content/haleiwa_03/159_3year.tif"
# input_tile_170 = "/content/haleiwa_03/170_3year.tif"
# input_tile_171 = "/content/haleiwa_03/171_3year.tif"
# input_tile_172 = "/content/haleiwa_03/172_3year.tif"
# input_tile_173 = "/content/haleiwa_03/173_3year.tif"
# input_tile_185 = "/content/haleiwa_03/185_3year.tif"
# input_tile_186 = "/content/haleiwa_03/186_3year.tif"
# input_tile_188 = "/content/haleiwa_03/188_3year.tif"
# input_tile_189 = "/content/haleiwa_03/189_3year.tif"
# input_tile_203 = "/content/haleiwa_03/203_3year.tif"
# input_tile_204 = "/content/haleiwa_03/204_3year.tif"
# input_tile_219 = "/content/haleiwa_03/219_3year.tif"
# input_tile_220 = "/content/haleiwa_03/220_3year.tif"
# input_tile_221 = "/content/haleiwa_03/221_3year.tif"
# input_tile_222 = "/content/haleiwa_03/222_3year.tif"
# input_tile_234 = "/content/haleiwa_03/234_3year.tif"
# input_tile_235 = "/content/haleiwa_03/235_3year.tif"
# input_tile_250 = "/content/haleiwa_03/250_3year.tif"
# input_tile_251 = "/content/haleiwa_03/251_3year.tif"
# input_tile_252 = "/content/haleiwa_03/252_3year.tif"
# input_tile_265 = "/content/haleiwa_03/265_3year.tif"
# input_tile_266 = "/content/haleiwa_03/266_3year.tif"
# input_tile_267 = "/content/haleiwa_03/267_3year.tif"
# input_tile_280 = "/content/haleiwa_03/280_3year.tif"
# input_tile_281 = "/content/haleiwa_03/281_3year.tif"
# input_tile_282 = "/content/haleiwa_03/282_3year.tif"
# input_tile_283 = "/content/haleiwa_03/283_3year.tif"
# input_tile_298 = "/content/haleiwa_03/298_3year.tif"
# input_tile_330 = "/content/haleiwa_03/330_3year.tif"


In [114]:
# input_tile_111 = "/content/haleiwa_02/111_2year.tif"
# input_tile_112 = "/content/haleiwa_02/112_2year.tif"
# input_tile_126 = "/content/haleiwa_02/126_2year.tif"
# input_tile_127 = "/content/haleiwa_02/127_2year.tif"
# input_tile_140 = "/content/haleiwa_02/140_2year.tif"
# input_tile_141 = "/content/haleiwa_02/141_2year.tif"
# input_tile_142 = "/content/haleiwa_02/142_2year.tif"
# input_tile_143 = "/content/haleiwa_02/143_2year.tif"
# input_tile_154 = "/content/haleiwa_02/154_2year.tif"
# input_tile_155 = "/content/haleiwa_02/155_2year.tif"
# input_tile_156 = "/content/haleiwa_02/156_2year.tif"
# input_tile_157 = "/content/haleiwa_02/157_2year.tif"
# input_tile_158 = "/content/haleiwa_02/158_2year.tif"
# input_tile_159 = "/content/haleiwa_02/159_2year.tif"
# input_tile_170 = "/content/haleiwa_02/170_2year.tif"
# input_tile_171 = "/content/haleiwa_02/171_2year.tif"
# input_tile_172 = "/content/haleiwa_02/172_2year.tif"
# input_tile_173 = "/content/haleiwa_02/173_2year.tif"
# input_tile_185 = "/content/haleiwa_02/185_2year.tif"
# input_tile_186 = "/content/haleiwa_02/186_2year.tif"
# input_tile_188 = "/content/haleiwa_02/188_2year.tif"
# input_tile_189 = "/content/haleiwa_02/189_2year.tif"
# input_tile_203 = "/content/haleiwa_02/203_2year.tif"
# input_tile_204 = "/content/haleiwa_02/204_2year.tif"
# input_tile_219 = "/content/haleiwa_02/219_2year.tif"
# input_tile_220 = "/content/haleiwa_02/220_2year.tif"
# input_tile_221 = "/content/haleiwa_02/221_2year.tif"
# input_tile_222 = "/content/haleiwa_02/222_2year.tif"
# input_tile_234 = "/content/haleiwa_02/234_2year.tif"
# input_tile_235 = "/content/haleiwa_02/235_2year.tif"
# input_tile_250 = "/content/haleiwa_02/250_2year.tif"
# input_tile_251 = "/content/haleiwa_02/251_2year.tif"
# input_tile_252 = "/content/haleiwa_02/252_2year.tif"
# input_tile_265 = "/content/haleiwa_02/265_2year.tif"
# input_tile_266 = "/content/haleiwa_02/266_2year.tif"
# input_tile_267 = "/content/haleiwa_02/267_2year.tif"
# input_tile_280 = "/content/haleiwa_02/280_2year.tif"
# input_tile_281 = "/content/haleiwa_02/281_2year.tif"
# input_tile_282 = "/content/haleiwa_02/282_2year.tif"
# input_tile_283 = "/content/haleiwa_02/283_2year.tif"
# input_tile_298 = "/content/haleiwa_02/298_2year.tif"
# input_tile_330 = "/content/haleiwa_02/330_2year.tif"


In [174]:
# # prompt: using this as an example, output_tile_111 = "/content/output_01/111_1year_out.tif"
# # change output_01 to output_02 for all files in this list.

# output_tile_111 = "/content/output_03/111_3year_out.tif"
# output_tile_112 = "/content/output_03/112_3year_out.tif"
# output_tile_126 = "/content/output_03/126_3year_out.tif"
# output_tile_127 = "/content/output_03/127_3year_out.tif"
# output_tile_140 = "/content/output_03/140_3year_out.tif"
# output_tile_141 = "/content/output_03/141_3year_out.tif"
# output_tile_142 = "/content/output_03/142_3year_out.tif"
# output_tile_143 = "/content/output_03/143_3year_out.tif"
# output_tile_154 = "/content/output_03/154_3year_out.tif"
# output_tile_155 = "/content/output_03/155_3year_out.tif"
# output_tile_156 = "/content/output_03/156_3year_out.tif"
# output_tile_157 = "/content/output_03/157_3year_out.tif"
# output_tile_158 = "/content/output_03/158_3year_out.tif"
# output_tile_159 = "/content/output_03/159_3year_out.tif"
# output_tile_170 = "/content/output_03/170_3year_out.tif"
# output_tile_171 = "/content/output_03/171_3year_out.tif"
# output_tile_172 = "/content/output_03/172_3year_out.tif"
# output_tile_173 = "/content/output_03/173_3year_out.tif"
# output_tile_185 = "/content/output_03/185_3year_out.tif"
# output_tile_186 = "/content/output_03/186_3year_out.tif"
# output_tile_188 = "/content/output_03/188_3year_out.tif"
# output_tile_189 = "/content/output_03/189_3year_out.tif"
# output_tile_203 = "/content/output_03/203_3year_out.tif"
# output_tile_204 = "/content/output_03/204_3year_out.tif"
# output_tile_219 = "/content/output_03/219_3year_out.tif"
# output_tile_220 = "/content/output_03/220_3year_out.tif"
# output_tile_221 = "/content/output_03/221_3year_out.tif"
# output_tile_222 = "/content/output_03/222_3year_out.tif"
# output_tile_234 = "/content/output_03/234_3year_out.tif"
# output_tile_235 = "/content/output_03/235_3year_out.tif"
# output_tile_250 = "/content/output_03/250_3year_out.tif"
# output_tile_251 = "/content/output_03/251_3year_out.tif"
# output_tile_252 = "/content/output_03/252_3year_out.tif"
# output_tile_265 = "/content/output_03/265_3year_out.tif"
# output_tile_266 = "/content/output_03/266_3year_out.tif"
# output_tile_267 = "/content/output_03/267_3year_out.tif"
# output_tile_280 = "/content/output_03/280_3year_out.tif"
# output_tile_281 = "/content/output_03/281_3year_out.tif"
# output_tile_282 = "/content/output_03/282_3year_out.tif"
# output_tile_283 = "/content/output_03/283_3year_out.tif"
# output_tile_298 = "/content/output_03/298_3year_out.tif"
# output_tile_330 = "/content/output_03/330_3year_out.tif"


In [115]:
# # prompt: using this as an example, output_tile_111 = "/content/output_01/111_1year_out.tif"
# # change output_01 to output_02 for all files in this list.

# output_tile_111 = "/content/output_02/111_2year_out.tif"
# output_tile_112 = "/content/output_02/112_2year_out.tif"
# output_tile_126 = "/content/output_02/126_2year_out.tif"
# output_tile_127 = "/content/output_02/127_2year_out.tif"
# output_tile_140 = "/content/output_02/140_2year_out.tif"
# output_tile_141 = "/content/output_02/141_2year_out.tif"
# output_tile_142 = "/content/output_02/142_2year_out.tif"
# output_tile_143 = "/content/output_02/143_2year_out.tif"
# output_tile_154 = "/content/output_02/154_2year_out.tif"
# output_tile_155 = "/content/output_02/155_2year_out.tif"
# output_tile_156 = "/content/output_02/156_2year_out.tif"
# output_tile_157 = "/content/output_02/157_2year_out.tif"
# output_tile_158 = "/content/output_02/158_2year_out.tif"
# output_tile_159 = "/content/output_02/159_2year_out.tif"
# output_tile_170 = "/content/output_02/170_2year_out.tif"
# output_tile_171 = "/content/output_02/171_2year_out.tif"
# output_tile_172 = "/content/output_02/172_2year_out.tif"
# output_tile_173 = "/content/output_02/173_2year_out.tif"
# output_tile_185 = "/content/output_02/185_2year_out.tif"
# output_tile_186 = "/content/output_02/186_2year_out.tif"
# output_tile_188 = "/content/output_02/188_2year_out.tif"
# output_tile_189 = "/content/output_02/189_2year_out.tif"
# output_tile_203 = "/content/output_02/203_2year_out.tif"
# output_tile_204 = "/content/output_02/204_2year_out.tif"
# output_tile_219 = "/content/output_02/219_2year_out.tif"
# output_tile_220 = "/content/output_02/220_2year_out.tif"
# output_tile_221 = "/content/output_02/221_2year_out.tif"
# output_tile_222 = "/content/output_02/222_2year_out.tif"
# output_tile_234 = "/content/output_02/234_2year_out.tif"
# output_tile_235 = "/content/output_02/235_2year_out.tif"
# output_tile_250 = "/content/output_02/250_2year_out.tif"
# output_tile_251 = "/content/output_02/251_2year_out.tif"
# output_tile_252 = "/content/output_02/252_2year_out.tif"
# output_tile_265 = "/content/output_02/265_2year_out.tif"
# output_tile_266 = "/content/output_02/266_2year_out.tif"
# output_tile_267 = "/content/output_02/267_2year_out.tif"
# output_tile_280 = "/content/output_02/280_2year_out.tif"
# output_tile_281 = "/content/output_02/281_2year_out.tif"
# output_tile_282 = "/content/output_02/282_2year_out.tif"
# output_tile_283 = "/content/output_02/283_2year_out.tif"
# output_tile_298 = "/content/output_02/298_2year_out.tif"
# output_tile_330 = "/content/output_02/330_2year_out.tif"


In [116]:
# # prompt: using this as an example, weight_tile_111 = "/content/output_weight_01/111_1year_weight.tif"
# # output_weight_01 to output_weight_02 for all files in this list.

# weight_tile_111 = "/content/output_weight_02/111_2year_weight.tif"
# weight_tile_112 = "/content/output_weight_02/112_2year_weight.tif"
# weight_tile_126 = "/content/output_weight_02/126_2year_weight.tif"
# weight_tile_127 = "/content/output_weight_02/127_2year_weight.tif"
# weight_tile_140 = "/content/output_weight_02/140_2year_weight.tif"
# weight_tile_141 = "/content/output_weight_02/141_2year_weight.tif"
# weight_tile_142 = "/content/output_weight_02/142_2year_weight.tif"
# weight_tile_143 = "/content/output_weight_02/143_2year_weight.tif"
# weight_tile_154 = "/content/output_weight_02/154_2year_weight.tif"
# weight_tile_155 = "/content/output_weight_02/155_2year_weight.tif"
# weight_tile_156 = "/content/output_weight_02/156_2year_weight.tif"
# weight_tile_157 = "/content/output_weight_02/157_2year_weight.tif"
# weight_tile_158 = "/content/output_weight_02/158_2year_weight.tif"
# weight_tile_159 = "/content/output_weight_02/159_2year_weight.tif"
# weight_tile_170 = "/content/output_weight_02/170_2year_weight.tif"
# weight_tile_171 = "/content/output_weight_02/171_2year_weight.tif"
# weight_tile_172 = "/content/output_weight_02/172_2year_weight.tif"
# weight_tile_173 = "/content/output_weight_02/173_2year_weight.tif"
# weight_tile_185 = "/content/output_weight_02/185_2year_weight.tif"
# weight_tile_186 = "/content/output_weight_02/186_2year_weight.tif"
# weight_tile_188 = "/content/output_weight_02/188_2year_weight.tif"
# weight_tile_189 = "/content/output_weight_02/189_2year_weight.tif"
# weight_tile_203 = "/content/output_weight_02/203_2year_weight.tif"
# weight_tile_204 = "/content/output_weight_02/204_2year_weight.tif"
# weight_tile_219 = "/content/output_weight_02/219_2year_weight.tif"
# weight_tile_220 = "/content/output_weight_02/220_2year_weight.tif"
# weight_tile_221 = "/content/output_weight_02/221_2year_weight.tif"
# weight_tile_222 = "/content/output_weight_02/222_2year_weight.tif"
# weight_tile_234 = "/content/output_weight_02/234_2year_weight.tif"
# weight_tile_235 = "/content/output_weight_02/235_2year_weight.tif"
# weight_tile_250 = "/content/output_weight_02/250_2year_weight.tif"
# weight_tile_251 = "/content/output_weight_02/251_2year_weight.tif"
# weight_tile_252 = "/content/output_weight_02/252_2year_weight.tif"
# weight_tile_265 = "/content/output_weight_02/265_2year_weight.tif"
# weight_tile_266 = "/content/output_weight_02/266_2year_weight.tif"
# weight_tile_267 = "/content/output_weight_02/267_2year_weight.tif"
# weight_tile_280 = "/content/output_weight_02/280_2year_weight.tif"
# weight_tile_281 = "/content/output_weight_02/281_2year_weight.tif"
# weight_tile_282 = "/content/output_weight_02/282_2year_weight.tif"
# weight_tile_283 = "/content/output_weight_02/283_2year_weight.tif"
# weight_tile_298 = "/content/output_weight_02/298_2year_weight.tif"
# weight_tile_330 = "/content/output_weight_02/330_2year_weight.tif"


In [64]:
# input_tile_111 = "/content/haleiwa_01/111_1year.tif"
# input_tile_112 = "/content/haleiwa_01/112_1year.tif"
# input_tile_126 = "/content/haleiwa_01/126_1year.tif"
# input_tile_127 = "/content/haleiwa_01/127_1year.tif"
# input_tile_140 = "/content/haleiwa_01/140_1year.tif"
# input_tile_141 = "/content/haleiwa_01/141_1year.tif"
# input_tile_142 = "/content/haleiwa_01/142_1year.tif"
# input_tile_143 = "/content/haleiwa_01/143_1year.tif"
# input_tile_154 = "/content/haleiwa_01/154_1year.tif"
# input_tile_155 = "/content/haleiwa_01/155_1year.tif"
# input_tile_156 = "/content/haleiwa_01/156_1year.tif"
# input_tile_157 = "/content/haleiwa_01/157_1year.tif"
# input_tile_158 = "/content/haleiwa_01/158_1year.tif"
# input_tile_159 = "/content/haleiwa_01/159_1year.tif"
# input_tile_170 = "/content/haleiwa_01/170_1year.tif"
# input_tile_171 = "/content/haleiwa_01/171_1year.tif"
# input_tile_172 = "/content/haleiwa_01/172_1year.tif"
# input_tile_173 = "/content/haleiwa_01/173_1year.tif"
# input_tile_185 = "/content/haleiwa_01/185_1year.tif"
# input_tile_186 = "/content/haleiwa_01/186_1year.tif"
# input_tile_188 = "/content/haleiwa_01/188_1year.tif"
# input_tile_189 = "/content/haleiwa_01/189_1year.tif"
# input_tile_203 = "/content/haleiwa_01/203_1year.tif"
# input_tile_204 = "/content/haleiwa_01/204_1year.tif"
# input_tile_219 = "/content/haleiwa_01/219_1year.tif"
# input_tile_220 = "/content/haleiwa_01/220_1year.tif"
# input_tile_221 = "/content/haleiwa_01/221_1year.tif"
# input_tile_222 = "/content/haleiwa_01/222_1year.tif"
# input_tile_234 = "/content/haleiwa_01/234_1year.tif"
# input_tile_235 = "/content/haleiwa_01/235_1year.tif"
# input_tile_250 = "/content/haleiwa_01/250_1year.tif"
# input_tile_251 = "/content/haleiwa_01/251_1year.tif"
# input_tile_252 = "/content/haleiwa_01/252_1year.tif"
# input_tile_265 = "/content/haleiwa_01/265_1year.tif"
# input_tile_266 = "/content/haleiwa_01/266_1year.tif"
# input_tile_267 = "/content/haleiwa_01/267_1year.tif"
# input_tile_280 = "/content/haleiwa_01/280_1year.tif"
# input_tile_281 = "/content/haleiwa_01/281_1year.tif"
# input_tile_282 = "/content/haleiwa_01/282_1year.tif"
# input_tile_283 = "/content/haleiwa_01/283_1year.tif"
# input_tile_298 = "/content/haleiwa_01/298_1year.tif"
# input_tile_330 = "/content/haleiwa_01/330_1year.tif"



# output_tile_111 = "/content/output_01/111_1year_out.tif"
# output_tile_112 = "/content/output_01/112_1year_out.tif"
# output_tile_126 = "/content/output_01/126_1year_out.tif"
# output_tile_127 = "/content/output_01/127_1year_out.tif"
# output_tile_140 = "/content/output_01/140_1year_out.tif"
# output_tile_141 = "/content/output_01/141_1year_out.tif"
# output_tile_142 = "/content/output_01/142_1year_out.tif"
# output_tile_143 = "/content/output_01/143_1year_out.tif"
# output_tile_154 = "/content/output_01/154_1year_out.tif"
# output_tile_155 = "/content/output_01/155_1year_out.tif"
# output_tile_156 = "/content/output_01/156_1year_out.tif"
# output_tile_157 = "/content/output_01/157_1year_out.tif"
# output_tile_158 = "/content/output_01/158_1year_out.tif"
# output_tile_159 = "/content/output_01/159_1year_out.tif"
# output_tile_170 = "/content/output_01/170_1year_out.tif"
# output_tile_171 = "/content/output_01/171_1year_out.tif"
# output_tile_172 = "/content/output_01/172_1year_out.tif"
# output_tile_173 = "/content/output_01/173_1year_out.tif"
# output_tile_185 = "/content/output_01/185_1year_out.tif"
# output_tile_186 = "/content/output_01/186_1year_out.tif"
# output_tile_188 = "/content/output_01/188_1year_out.tif"
# output_tile_189 = "/content/output_01/189_1year_out.tif"
# output_tile_203 = "/content/output_01/203_1year_out.tif"
# output_tile_204 = "/content/output_01/204_1year_out.tif"
# output_tile_219 = "/content/output_01/219_1year_out.tif"
# output_tile_220 = "/content/output_01/220_1year_out.tif"
# output_tile_221 = "/content/output_01/221_1year_out.tif"
# output_tile_222 = "/content/output_01/222_1year_out.tif"
# output_tile_234 = "/content/output_01/234_1year_out.tif"
# output_tile_235 = "/content/output_01/235_1year_out.tif"
# output_tile_250 = "/content/output_01/250_1year_out.tif"
# output_tile_251 = "/content/output_01/251_1year_out.tif"
# output_tile_252 = "/content/output_01/252_1year_out.tif"
# output_tile_265 = "/content/output_01/265_1year_out.tif"
# output_tile_266 = "/content/output_01/266_1year_out.tif"
# output_tile_267 = "/content/output_01/267_1year_out.tif"
# output_tile_280 = "/content/output_01/280_1year_out.tif"
# output_tile_281 = "/content/output_01/281_1year_out.tif"
# output_tile_282 = "/content/output_01/282_1year_out.tif"
# output_tile_283 = "/content/output_01/283_1year_out.tif"
# output_tile_298 = "/content/output_01/298_1year_out.tif"
# output_tile_330 = "/content/output_01/330_1year_out.tif"



# weight_tile_111 = "/content/output_weight_01/111_1year_weight.tif"
# weight_tile_112 = "/content/output_weight_01/112_1year_weight.tif"
# weight_tile_126 = "/content/output_weight_01/126_1year_weight.tif"
# weight_tile_127 = "/content/output_weight_01/127_1year_weight.tif"
# weight_tile_140 = "/content/output_weight_01/140_1year_weight.tif"
# weight_tile_141 = "/content/output_weight_01/141_1year_weight.tif"
# weight_tile_142 = "/content/output_weight_01/142_1year_weight.tif"
# weight_tile_143 = "/content/output_weight_01/143_1year_weight.tif"
# weight_tile_154 = "/content/output_weight_01/154_1year_weight.tif"
# weight_tile_155 = "/content/output_weight_01/155_1year_weight.tif"
# weight_tile_156 = "/content/output_weight_01/156_1year_weight.tif"
# weight_tile_157 = "/content/output_weight_01/157_1year_weight.tif"
# weight_tile_158 = "/content/output_weight_01/158_1year_weight.tif"
# weight_tile_159 = "/content/output_weight_01/159_1year_weight.tif"
# weight_tile_170 = "/content/output_weight_01/170_1year_weight.tif"
# weight_tile_171 = "/content/output_weight_01/171_1year_weight.tif"
# weight_tile_172 = "/content/output_weight_01/172_1year_weight.tif"
# weight_tile_173 = "/content/output_weight_01/173_1year_weight.tif"
# weight_tile_185 = "/content/output_weight_01/185_1year_weight.tif"
# weight_tile_186 = "/content/output_weight_01/186_1year_weight.tif"
# weight_tile_188 = "/content/output_weight_01/188_1year_weight.tif"
# weight_tile_189 = "/content/output_weight_01/189_1year_weight.tif"
# weight_tile_203 = "/content/output_weight_01/203_1year_weight.tif"
# weight_tile_204 = "/content/output_weight_01/204_1year_weight.tif"
# weight_tile_219 = "/content/output_weight_01/219_1year_weight.tif"
# weight_tile_220 = "/content/output_weight_01/220_1year_weight.tif"
# weight_tile_221 = "/content/output_weight_01/221_1year_weight.tif"
# weight_tile_222 = "/content/output_weight_01/222_1year_weight.tif"
# weight_tile_234 = "/content/output_weight_01/234_1year_weight.tif"
# weight_tile_235 = "/content/output_weight_01/235_1year_weight.tif"
# weight_tile_250 = "/content/output_weight_01/250_1year_weight.tif"
# weight_tile_251 = "/content/output_weight_01/251_1year_weight.tif"
# weight_tile_252 = "/content/output_weight_01/252_1year_weight.tif"
# weight_tile_265 = "/content/output_weight_01/265_1year_weight.tif"
# weight_tile_266 = "/content/output_weight_01/266_1year_weight.tif"
# weight_tile_267 = "/content/output_weight_01/267_1year_weight.tif"
# weight_tile_280 = "/content/output_weight_01/280_1year_weight.tif"
# weight_tile_281 = "/content/output_weight_01/281_1year_weight.tif"
# weight_tile_282 = "/content/output_weight_01/282_1year_weight.tif"
# weight_tile_283 = "/content/output_weight_01/283_1year_weight.tif"
# weight_tile_298 = "/content/output_weight_01/298_1year_weight.tif"
# weight_tile_330 = "/content/output_weight_01/330_1year_weight.tif"



In [ ]:
# tile_742 tile_743 tile_759 tile_760 tile_761 tile_762 tile_776 tile_777 tile_778 tile_779 tile_780 tile_781
# tile_792 tile_793 tile_794 tile_795 tile_796 tile_797 tile_798 tile_805 tile_808 tile_809 tile_810 tile_811
# tile_812 tile_813 tile_815 tile_821 tile_823 tile_825 tile_826 tile_827 tile_828 tile_829 tile_830 tile_831
# tile_841 tile_842 tile_843 tile_845 tile_847 tile_848 tile_849 tile_860 tile_861 tile_862 tile_865 tile_866

In [228]:
input_tile_742 = "/content/kahuku_3year/742_3year.tif"
output_tile_742 = "/content/kahuku_output_03/742_3year_out.tif"
weight_tile_742 = "/content/kahuku_weight_03/742_3year_weight.tif"

input_tile_743 = "/content/kahuku_3year/743_3year.tif"
output_tile_743 = "/content/kahuku_output_03/743_3year_out.tif"
weight_tile_743 = "/content/kahuku_weight_03/743_3year_weight.tif"

input_tile_759 = "/content/kahuku_3year/759_3year.tif"
output_tile_759 = "/content/kahuku_output_03/759_3year_out.tif"
weight_tile_759 = "/content/kahuku_weight_03/759_3year_weight.tif"

input_tile_760 = "/content/kahuku_3year/760_3year.tif"
output_tile_760 = "/content/kahuku_output_03/760_3year_out.tif"
weight_tile_760 = "/content/kahuku_weight_03/760_3year_weight.tif"

input_tile_761 = "/content/kahuku_3year/761_3year.tif"
output_tile_761 = "/content/kahuku_output_03/761_3year_out.tif"
weight_tile_761 = "/content/kahuku_weight_03/761_3year_weight.tif"

input_tile_762 = "/content/kahuku_3year/762_3year.tif"
output_tile_762 = "/content/kahuku_output_03/762_3year_out.tif"
weight_tile_762 = "/content/kahuku_weight_03/762_3year_weight.tif"

input_tile_776 = "/content/kahuku_3year/776_3year.tif"
output_tile_776 = "/content/kahuku_output_03/776_3year_out.tif"
weight_tile_776 = "/content/kahuku_weight_03/776_3year_weight.tif"

input_tile_777 = "/content/kahuku_3year/777_3year.tif"
output_tile_777 = "/content/kahuku_output_03/777_3year_out.tif"
weight_tile_777 = "/content/kahuku_weight_03/777_3year_weight.tif"

input_tile_778 = "/content/kahuku_3year/778_3year.tif"
output_tile_778 = "/content/kahuku_output_03/778_3year_out.tif"
weight_tile_778 = "/content/kahuku_weight_03/778_3year_weight.tif"

input_tile_779 = "/content/kahuku_3year/779_3year.tif"
output_tile_779 = "/content/kahuku_output_03/779_3year_out.tif"
weight_tile_779 = "/content/kahuku_weight_03/779_3year_weight.tif"

input_tile_780 = "/content/kahuku_3year/780_3year.tif"
output_tile_780 = "/content/kahuku_output_03/780_3year_out.tif"
weight_tile_780 = "/content/kahuku_weight_03/780_3year_weight.tif"

input_tile_781 = "/content/kahuku_3year/781_3year.tif"
output_tile_781 = "/content/kahuku_output_03/781_3year_out.tif"
weight_tile_781 = "/content/kahuku_weight_03/781_3year_weight.tif"

input_tile_792 = "/content/kahuku_3year/792_3year.tif"
output_tile_792 = "/content/kahuku_output_03/792_3year_out.tif"
weight_tile_792 = "/content/kahuku_weight_03/792_3year_weight.tif"

input_tile_793 = "/content/kahuku_3year/793_3year.tif"
output_tile_793 = "/content/kahuku_output_03/793_3year_out.tif"
weight_tile_793 = "/content/kahuku_weight_03/793_3year_weight.tif"

input_tile_794 = "/content/kahuku_3year/794_3year.tif"
output_tile_794 = "/content/kahuku_output_03/794_3year_out.tif"
weight_tile_794 = "/content/kahuku_weight_03/794_3year_weight.tif"

input_tile_795 = "/content/kahuku_3year/795_3year.tif"
output_tile_795 = "/content/kahuku_output_03/795_3year_out.tif"
weight_tile_795 = "/content/kahuku_weight_03/795_3year_weight.tif"

input_tile_796 = "/content/kahuku_3year/796_3year.tif"
output_tile_796 = "/content/kahuku_output_03/796_3year_out.tif"
weight_tile_796 = "/content/kahuku_weight_03/796_3year_weight.tif"

input_tile_797 = "/content/kahuku_3year/797_3year.tif"
output_tile_797 = "/content/kahuku_output_03/797_3year_out.tif"
weight_tile_797 = "/content/kahuku_weight_03/797_3year_weight.tif"

input_tile_798 = "/content/kahuku_3year/798_3year.tif"
output_tile_798 = "/content/kahuku_output_03/798_3year_out.tif"
weight_tile_798 = "/content/kahuku_weight_03/798_3year_weight.tif"

input_tile_805 = "/content/kahuku_3year/805_3year.tif"
output_tile_805 = "/content/kahuku_output_03/805_3year_out.tif"
weight_tile_805 = "/content/kahuku_weight_03/805_3year_weight.tif"

input_tile_808 = "/content/kahuku_3year/808_3year.tif"
output_tile_808 = "/content/kahuku_output_03/808_3year_out.tif"
weight_tile_808 = "/content/kahuku_weight_03/808_3year_weight.tif"

input_tile_809 = "/content/kahuku_3year/809_3year.tif"
output_tile_809 = "/content/kahuku_output_03/809_3year_out.tif"
weight_tile_809 = "/content/kahuku_weight_03/809_3year_weight.tif"

input_tile_810 = "/content/kahuku_3year/810_3year.tif"
output_tile_810 = "/content/kahuku_output_03/810_3year_out.tif"
weight_tile_810 = "/content/kahuku_weight_03/810_3year_weight.tif"

input_tile_811 = "/content/kahuku_3year/811_3year.tif"
output_tile_811 = "/content/kahuku_output_03/811_3year_out.tif"
weight_tile_811 = "/content/kahuku_weight_03/811_3year_weight.tif"

input_tile_812 = "/content/kahuku_3year/812_3year.tif"
output_tile_812 = "/content/kahuku_output_03/812_3year_out.tif"
weight_tile_812 = "/content/kahuku_weight_03/812_3year_weight.tif"

input_tile_813 = "/content/kahuku_3year/813_3year.tif"
output_tile_813 = "/content/kahuku_output_03/813_3year_out.tif"
weight_tile_813 = "/content/kahuku_weight_03/813_3year_weight.tif"

input_tile_815 = "/content/kahuku_3year/815_3year.tif"
output_tile_815 = "/content/kahuku_output_03/815_3year_out.tif"
weight_tile_815 = "/content/kahuku_weight_03/815_3year_weight.tif"

input_tile_821 = "/content/kahuku_3year/821_3year.tif"
output_tile_821 = "/content/kahuku_output_03/821_3year_out.tif"
weight_tile_821 = "/content/kahuku_weight_03/821_3year_weight.tif"

input_tile_823 = "/content/kahuku_3year/823_3year.tif"
output_tile_823 = "/content/kahuku_output_03/823_3year_out.tif"
weight_tile_823 = "/content/kahuku_weight_03/823_3year_weight.tif"

input_tile_825 = "/content/kahuku_3year/825_3year.tif"
output_tile_825 = "/content/kahuku_output_03/825_3year_out.tif"
weight_tile_825 = "/content/kahuku_weight_03/825_3year_weight.tif"

input_tile_826 = "/content/kahuku_3year/826_3year.tif"
output_tile_826 = "/content/kahuku_output_03/826_3year_out.tif"
weight_tile_826 = "/content/kahuku_weight_03/826_3year_weight.tif"

input_tile_827 = "/content/kahuku_3year/827_3year.tif"
output_tile_827 = "/content/kahuku_output_03/827_3year_out.tif"
weight_tile_827 = "/content/kahuku_weight_03/827_3year_weight.tif"

input_tile_828 = "/content/kahuku_3year/828_3year.tif"
output_tile_828 = "/content/kahuku_output_03/828_3year_out.tif"
weight_tile_828 = "/content/kahuku_weight_03/828_3year_weight.tif"

input_tile_829 = "/content/kahuku_3year/829_3year.tif"
output_tile_829 = "/content/kahuku_output_03/829_3year_out.tif"
weight_tile_829 = "/content/kahuku_weight_03/829_3year_weight.tif"

input_tile_830 = "/content/kahuku_3year/830_3year.tif"
output_tile_830 = "/content/kahuku_output_03/830_3year_out.tif"
weight_tile_830 = "/content/kahuku_weight_03/830_3year_weight.tif"

input_tile_831 = "/content/kahuku_3year/831_3year.tif"
output_tile_831 = "/content/kahuku_output_03/831_3year_out.tif"
weight_tile_831 = "/content/kahuku_weight_03/831_3year_weight.tif"

input_tile_841 = "/content/kahuku_3year/841_3year.tif"
output_tile_841 = "/content/kahuku_output_03/841_3year_out.tif"
weight_tile_841 = "/content/kahuku_weight_03/841_3year_weight.tif"

input_tile_842 = "/content/kahuku_3year/842_3year.tif"
output_tile_842 = "/content/kahuku_output_03/842_3year_out.tif"
weight_tile_842 = "/content/kahuku_weight_03/842_3year_weight.tif"

input_tile_843 = "/content/kahuku_3year/843_3year.tif"
output_tile_843 = "/content/kahuku_output_03/843_3year_out.tif"
weight_tile_843 = "/content/kahuku_weight_03/843_3year_weight.tif"

input_tile_845 = "/content/kahuku_3year/845_3year.tif"
output_tile_845 = "/content/kahuku_output_03/845_3year_out.tif"
weight_tile_845 = "/content/kahuku_weight_03/845_3year_weight.tif"

input_tile_847 = "/content/kahuku_3year/847_3year.tif"
output_tile_847 = "/content/kahuku_output_03/847_3year_out.tif"
weight_tile_847 = "/content/kahuku_weight_03/847_3year_weight.tif"

input_tile_848 = "/content/kahuku_3year/848_3year.tif"
output_tile_848 = "/content/kahuku_output_03/848_3year_out.tif"
weight_tile_848 = "/content/kahuku_weight_03/848_3year_weight.tif"

input_tile_849 = "/content/kahuku_3year/849_3year.tif"
output_tile_849 = "/content/kahuku_output_03/849_3year_out.tif"
weight_tile_849 = "/content/kahuku_weight_03/849_3year_weight.tif"

input_tile_860 = "/content/kahuku_3year/860_3year.tif"
output_tile_860 = "/content/kahuku_output_03/860_3year_out.tif"
weight_tile_860 = "/content/kahuku_weight_03/860_3year_weight.tif"

input_tile_861 = "/content/kahuku_3year/861_3year.tif"
output_tile_861 = "/content/kahuku_output_03/861_3year_out.tif"
weight_tile_861 = "/content/kahuku_weight_03/861_3year_weight.tif"

input_tile_862 = "/content/kahuku_3year/862_3year.tif"
output_tile_862 = "/content/kahuku_output_03/862_3year_out.tif"
weight_tile_862 = "/content/kahuku_weight_03/862_3year_weight.tif"

input_tile_865 = "/content/kahuku_3year/865_3year.tif"
output_tile_865 = "/content/kahuku_output_03/865_3year_out.tif"
weight_tile_865 = "/content/kahuku_weight_03/865_3year_weight.tif"

input_tile_866 = "/content/kahuku_3year/866_3year.tif"
output_tile_866 = "/content/kahuku_output_03/866_3year_out.tif"
weight_tile_866 = "/content/kahuku_weight_03/866_3year_weight.tif"

In [229]:
# prompt: make new folder "kahuku_output_03"
# make new folder "kahuku_weight_03"

!mkdir kahuku_output_03
!mkdir kahuku_weight_03


In [232]:

!mkdir kahuku_output_03
!mkdir kahuku_weight_03

tiles = [742, 743, 759, 760, 761, 762, 776, 777, 778, 779, 780, 781, 792, 793, 794, 795, 796, 797, 798, 805, 808, 809, 810, 811, 812, 813, 815, 821, 823, 825, 826, 827, 828, 829, 830, 831, 841, 842, 843, 845, 847, 848, 849, 860, 861, 862, 865, 866]

for tile in tiles:
    input_tile = f"/content/kahuku_3year/{tile}_3year.tif"
    output_tile = f"/content/kahuku_output_03/{tile}_3year_out.tif"
    weight_tile = f"/content/kahuku_weight_03/{tile}_3year_weight.tif"

    tile_result = eval(f'tile_{tile}_result')

    with rasterio.open(input_tile) as src:
        out_meta = src.meta.copy()
        out_meta.update({
            'dtype': 'float32',
            'count' : 7
        })
        data_out = tile_result[0]
        data_out = np.moveaxis(data_out, -1, 0)
        weight_matrix = create_weight_matrix(384, 384)
        weighted_data_out = weight_matrix * data_out

    with rasterio.open(output_tile, 'w', **out_meta) as dst:
        dst.write(data_out)
    with rasterio.open(weight_tile, 'w', **out_meta) as dst:
        dst.write(weighted_data_out)


In [233]:
import os
import rasterio
from rasterio.merge import merge

def gather_raster_files(*directories):
    """Gather raster files from provided directories."""
    raster_paths = []
    for directory in directories:
        # List all TIFF files in each directory
        for filename in os.listdir(directory):
            if filename.endswith('.tif'):
                raster_paths.append(os.path.join(directory, filename))
    return raster_paths

def create_mosaic_sum(input_rasters, output_filename='mosaic_sum.tif'):
    """Create a mosaic from the sum of overlapping bands using rasterio."""
    src_files = [rasterio.open(path) for path in input_rasters]

    # Use rasterio's merge function with method='sum' to sum overlapping pixels
    mosaic, out_trans = merge(src_files, method='sum')

    # Define the metadata for the output GeoTIFF
    out_meta = src_files[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "count": src_files[0].count
    })

    # Write the mosaic raster to disk
    with rasterio.open(output_filename, 'w', **out_meta) as dest:
        dest.write(mosaic)

    # Close all raster files
    for src in src_files:
        src.close()

    print(f"Mosaic created successfully: {output_filename}")
    return mosaic, out_trans

# Directories containing the raster files
output_dir = "/content/kahuku_output_03"
weighted_dir = "/content/kahuku_weight_03"

# Gather all raster files from the directories
raster_files = gather_raster_files(output_dir, weighted_dir)

# Create a mosaic with the sum of overlapping bands
mosaic, out_transform = create_mosaic_sum(raster_files, 'kahuku_03_mosaic.tif')

Mosaic created successfully: kahuku_03_mosaic.tif


In [175]:
with rasterio.open(input_tile_111) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_111_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_111, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_111, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [176]:
with rasterio.open(input_tile_112) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_112_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_112, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_112, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [177]:
with rasterio.open(input_tile_126) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_126_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_126, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_126, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [178]:
with rasterio.open(input_tile_127) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_127_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_127, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_127, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [179]:
with rasterio.open(input_tile_140) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_140_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_140, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_140, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [180]:
with rasterio.open(input_tile_141) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_141_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_141, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_141, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [181]:
with rasterio.open(input_tile_142) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_142_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_142, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_142, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [182]:
with rasterio.open(input_tile_143) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_143_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_143, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_143, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [183]:
with rasterio.open(input_tile_154) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_154_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_154, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_154, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [184]:
with rasterio.open(input_tile_155) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_155_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_155, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_155, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [185]:
with rasterio.open(input_tile_156) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_156_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_156, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_156, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [186]:
with rasterio.open(input_tile_157) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_157_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_157, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_157, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)


In [187]:
with rasterio.open(input_tile_158) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_158_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_158, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_158, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [188]:
with rasterio.open(input_tile_159) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_159_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

  with rasterio.open(output_tile_159, 'w', **out_meta) as dst:
    dst.write(data_out)
  with rasterio.open(weight_tile_159, 'w', **out_meta) as dst:
    dst.write(weighted_data_out)

In [189]:
with rasterio.open(input_tile_170) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_170_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_170, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_170, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [190]:
with rasterio.open(input_tile_171) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_171_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_171, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_171, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [191]:
with rasterio.open(input_tile_172) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_172_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out


with rasterio.open(output_tile_172, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_172, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [192]:
with rasterio.open(input_tile_173) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_173_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out


with rasterio.open(output_tile_173, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_173, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [193]:
with rasterio.open(input_tile_185) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_185_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out


with rasterio.open(output_tile_185, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_185, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [194]:
with rasterio.open(input_tile_186) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_186_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_186, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_186, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [195]:
with rasterio.open(input_tile_188) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_188_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_188, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_188, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [196]:
with rasterio.open(input_tile_189) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_189_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_189, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_189, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [197]:
with rasterio.open(input_tile_203) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_203_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_203, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_203, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [198]:
with rasterio.open(input_tile_204) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_204_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_204, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_204, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [199]:
with rasterio.open(input_tile_219) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_219_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_219, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_219, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [200]:
with rasterio.open(input_tile_220) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_220_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_220, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_220, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [201]:
with rasterio.open(input_tile_221) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_221_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_221, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_221, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [202]:
with rasterio.open(input_tile_222) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_222_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_222, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_222, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [203]:

with rasterio.open(input_tile_234) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_234_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_234, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_234, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [204]:
with rasterio.open(input_tile_235) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_235_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_235, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_235, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [205]:
with rasterio.open(input_tile_250) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_250_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_250, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_250, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [206]:
with rasterio.open(input_tile_251) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_251_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_251, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_251, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [207]:
with rasterio.open(input_tile_252) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_252_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_252, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_252, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [208]:
with rasterio.open(input_tile_265) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_265_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_265, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_265, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [209]:
with rasterio.open(input_tile_266) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_266_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_266, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_266, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [210]:
with rasterio.open(input_tile_267) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_267_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_267, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_267, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [211]:
with rasterio.open(input_tile_280) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_280_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_280, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_280, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [212]:
with rasterio.open(input_tile_281) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_281_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_281, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_281, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [213]:
with rasterio.open(input_tile_282) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_282_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_282, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_282, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [214]:
with rasterio.open(input_tile_283) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_283_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_283, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_283, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [215]:
with rasterio.open(input_tile_298) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_298_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_298, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_298, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [216]:
with rasterio.open(input_tile_330) as src:
  out_meta = src.meta.copy()
  out_meta.update({
      'dytpe': 'float32',
      'count' : 7
  })
  data_out = tile_330_result[0]
  data_out = np.moveaxis(data_out,-1,0)
  weight_matrix = create_weight_matrix(384,384)
  weighted_data_out = weight_matrix*data_out

with rasterio.open(output_tile_330, 'w', **out_meta) as dst:
  dst.write(data_out)
with rasterio.open(weight_tile_330, 'w', **out_meta) as dst:
  dst.write(weighted_data_out)

In [217]:
directory_path = "/content/output_weight_03"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

There are 42 files in /content/output_weight_03.


In [218]:
fgdsgr4ew3t4324554543

NameError: name 'fgdsgr4ew3t4324554543' is not defined

In [219]:
import os
import rasterio
from rasterio.merge import merge

def gather_raster_files(*directories):
    """Gather raster files from provided directories."""
    raster_paths = []
    for directory in directories:
        # List all TIFF files in each directory
        for filename in os.listdir(directory):
            if filename.endswith('.tif'):
                raster_paths.append(os.path.join(directory, filename))
    return raster_paths

def create_mosaic_sum(input_rasters, output_filename='mosaic_sum.tif'):
    """Create a mosaic from the sum of overlapping bands using rasterio."""
    src_files = [rasterio.open(path) for path in input_rasters]

    # Use rasterio's merge function with method='sum' to sum overlapping pixels
    mosaic, out_trans = merge(src_files, method='sum')

    # Define the metadata for the output GeoTIFF
    out_meta = src_files[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "count": src_files[0].count
    })

    # Write the mosaic raster to disk
    with rasterio.open(output_filename, 'w', **out_meta) as dest:
        dest.write(mosaic)

    # Close all raster files
    for src in src_files:
        src.close()

    print(f"Mosaic created successfully: {output_filename}")
    return mosaic, out_trans

# Directories containing the raster files
output_dir = "/content/output_03"
weighted_dir = "/content/output_weight_03"

# Gather all raster files from the directories
raster_files = gather_raster_files(output_dir, weighted_dir)

# Create a mosaic with the sum of overlapping bands
mosaic, out_transform = create_mosaic_sum(raster_files, 'haleiwa_03_mosaic.tif')


Mosaic created successfully: haleiwa_03_mosaic.tif


In [ ]:
def create_weight_matrix(height, width):
    center_x, center_y = width // 2, height // 2
    Y, X = np.ogrid[:height, :width]
    distance_from_center = np.sqrt((X - center_x)**2 + (Y - center_y)**2)

    # Normalize the distance matrix so that the maximum distance is 1
    max_distance = np.sqrt(center_x**2 + center_y**2)
    normalized_distance = distance_from_center / max_distance

    # Create a weight matrix where weights linearly decrease from 1 at the center to 0 at the edges
    weight_matrix = 1 - normalized_distance

    # Normalization for overlaps was incorrect in the initial approach, let's correct it:
    # The matrix already ensures the center has weight 1.
    # To accommodate overlapping sum to 1, no further division is required assuming
    # that the sum of any two overlapping areas naturally scales with this setup.

    return weight_matrix

# Example usage:
height, width = 256, 256  # Adjust as needed
weight_matrix = create_weight_matrix(height, width)